# Top-down modulation of illusory contours paper
### see https://www.jneurosci.org/content/40/3/648

In [ ]:
# import libraries
from __future__ import division
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator
import scipy.stats as sstat
import scipy.signal as ssig
import h5py
from mpl_toolkits.mplot3d import Axes3D
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA
import re
import ephys_unit_analysis as ena
import fnmatch
# from PyPDF2 import PdfFileMerger, PdfFileReader
import seaborn as sns
%matplotlib inline
mpl.rcParams['pdf.fonttype'] = 42 
mpl.rcParams['font.sans-serif']=['Arial', 'Helvetica','Bitstream Vera Sans', 'DejaVu Sans', 'Lucida Grande', 
                                 'Verdana', 'Geneva', 'Lucid', 'Avant Garde', 'sans-serif']  

%load_ext autoreload
%autoreload 2
# pal=sns.blend_palette(["black", "crimson"], 2)
sns.despine()
# current_palette = sns.color_palette("colorblind", 10)
# sns.set_palette(current_palette)

# for publication quality plots, not bar graphs, use this: 
def set_pub_plots(pal=sns.blend_palette(["crimson","gray", 'cyan',  'purple', 'magenta' ],5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":2.5, 'figure.facecolor': 'white'}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    # optional, makes markers bigger, too, axes.linewidth doesn't seem to work
    plt.rcParams['axes.linewidth'] = 2.5

rc_pub={'font.size': 25, 'axes.labelsize': 25, 'legend.fontsize': 25.0, 
    'axes.titlesize': 25, 'xtick.labelsize': 25, 'ytick.labelsize': 25, 
    #'axes.color_cycle':pal, # image.cmap - rewritesd the default colormap
    'axes.linewidth':2.5, 'lines.linewidth': 2.5,
    'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black','axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

#set_pub_bargraphs()
set_pub_plots()



#%matplotlib notebook

In [ ]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

In [ ]:
# define color palette
my_pal = np.zeros((8,3))
colors = sns.color_palette("Paired_r", 12)
colors = np.array(colors)*0.9
my_pal[0] = colors[6]
my_pal[1] = colors[7]
my_pal[2] = colors[8]
my_pal[3] = colors[9]
my_pal[4] = colors[10]
my_pal[5] = colors[11]
my_pal[6] = colors[2]
my_pal[7] = colors[3]
colors = my_pal[:]
# colors = colors[::2]
sns.palplot(colors)

In [ ]:
# extract frame indices from frames
# generate STA RF maps
def avg_rf_on_off(_input, d_rf, et, rec):
    d_frames_idx = {}
    df_rez = _input
    for unit in sorted(df_rez.cluster_id.unique()[:]):
        cluster_id = str(unit) + '005' + 'et' + str(et) + str(rec) 
        tmp = df_rez[df_rez.cluster_id == unit]
        # account for latency and 120 empty frames, convert to frames idx
        tmp =  np.rint((tmp.times.values[:] + 0.05 -2)*60)

    #     frame_idx = [round_to_frame(x) for x in tmp]
        frame_idx = ((tmp//15))
        frame_idx = frame_idx[frame_idx>-1]
        frame_idx = frame_idx[frame_idx < 3002]
        frame_idx = frame_idx.astype(int)

        unique, counts = np.unique(frame_idx, return_counts=True)
        d_frames_idx = dict(zip(unique, counts))
        avg_frame_on = np.average(frames_arr_on[d_frames_idx.keys()],axis = 0, weights=d_frames_idx.values())
        avg_frame_off = np.average(frames_arr_off[d_frames_idx.keys()],axis = 0, weights=d_frames_idx.values())
        d_rf['on'][cluster_id] = avg_frame_on
        d_rf['off'][cluster_id] = avg_frame_off
    return d_rf

In [ ]:
# monitor specs
from math import atan2, degrees
mon_resolution = (1080,1920) # vert, horiz
mon_width_cm = 48.1 #enter your monitors width in cm
mon_height_cm = 28.5 #enter your monitors height in cm

mon_width_cm = 59.7 #VS big monitor
mon_height_cm = 33.5 #VS big monitor

monitor_distance = 17 # cm

probe_in_px = 48 #lsn probe/square size
cir_r = 100 #opto experiments
sqr = 600 # opto exp

# sqr = 400 # kic-sqr-ori
# cir_r = 80 # kic-sqr-ori experiments

line_w = 10
kic = sqr - 2*cir_r
deg_per_px = degrees(atan2(.5*mon_height_cm, monitor_distance)) / (.5*mon_resolution[0])
print '%s degrees correspond to a single pixel' % deg_per_px
probe_in_deg = probe_in_px * deg_per_px
cir_r_deg = cir_r * deg_per_px
sqr_deg = sqr * deg_per_px
kic_deg = kic * deg_per_px
line_w_deg = line_w * deg_per_px
print 'The size of the stimulus is %s pixels and %s visual degrees' \
    % (probe_in_px, probe_in_deg)
print 'rad of circular disc', cir_r_deg
print 'sqr', sqr_deg, 'kic', kic_deg
print 'line width', line_w_deg

In [ ]:
# dics to map stimuli to trials
trial_seq = [3, 3, 1, 4, 3, 4, 4, 1, 4, 4, 2, 1, 4, 4, 3, 4, 4, 3, 2, 4, 3, 2, 3, 4, 4, 2, 2, 3, 4, 4, 4, 4, 2, 2, 3, 1, 2, 3, 1, 1, 2, 4, 2, 4, 1, 2, 4, 2, 4, 4, 3, 2, 4, 3, 3, 2, 3, 2, 1, 1, 4, 4, 2, 4, 4, 3, 2, 3, 1, 1, 3, 2, 2, 1, 1, 1, 1, 1, 4, 2, 4, 2, 3, 3, 2, 1, 3, 3, 2, 1, 3, 1, 2, 3, 4, 4, 2, 4, 1, 3, 2, 1, 1, 4, 4, 3, 1, 3, 3, 4, 1, 1, 3, 3, 3, 2, 3, 1, 1, 4, 2, 1, 4, 4, 4, 2, 2, 2, 4, 3, 4, 3, 2, 1, 1, 2, 1, 4, 3, 4, 3, 2, 3, 1, 3, 1, 2, 3, 2, 1, 1, 4, 1, 1, 1, 1, 3, 2, 1, 1, 3, 3, 3, 2, 1, 1, 1, 3, 2, 2, 3, 1, 4, 2, 1, 2, 4, 1, 3, 2, 4, 4, 3, 2, 4, 2, 3, 2, 1, 4, 3, 3, 1, 3, 2, 2, 1, 2, 2, 4]
opto_idx = [1, 2, 3, 5, 9, 10, 12, 15, 21, 24, 25, 28, 29, 31, 32, 34, 35, 36, 41, 43, 44, 45, 47, 48, 49, 50, 51, 52, 54, 57, 62, 64, 67, 68, 71, 76, 79, 80, 81, 84, 86, 87, 88, 91, 92, 96, 98, 99, 102, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 120, 121, 122, 128, 129, 131, 133, 138, 139, 140, 142, 143, 144, 147, 148, 149, 150, 151, 153, 155, 158, 159, 161, 162, 163, 165, 170, 172, 173, 177, 181, 183, 186, 188, 189, 190, 192, 194, 196, 197, 198]
trial_seq = np.array(trial_seq)
short_seq = np.array(trial_seq[opto_idx])
opto_seq = [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0]
opto_seq = np.array(opto_seq)
dir_12 = np.array([10, 7, 3, 2, 4, 8, 9, 5, 7, 3, 4, 8, 3, 2, 1, 8, 0, 4, 9, 11, 
10, 9, 1, 11, 4, 0, 7, 1, 2, 8, 2, 9, 11, 9, 6, 5, 10, 4, 9, 0, 7, 11, 9, 
5, 9, 10, 11, 6, 8, 9, 5, 4, 2, 8, 11, 2, 10, 3, 5, 1, 7, 0, 4, 9, 1, 5, 
11, 3, 5, 10, 1, 2, 9, 6, 2, 2, 11, 5, 10, 7, 3, 7, 4, 6, 8, 4, 1, 8, 0, 
11, 0, 6, 2, 11, 1, 10, 3, 8, 3, 1, 2, 10, 5, 3, 11, 1, 7, 3, 4, 7, 8, 4, 6, 
7, 11, 7, 0, 8, 6, 10, 4, 5, 7, 2, 10, 3, 5, 9, 8, 6, 3, 2, 0, 11, 0, 6, 10, 
0, 7, 4, 5, 0, 10, 6, 8, 10, 3, 11, 9, 0, 5, 1, 3, 7, 0, 6, 9, 1, 6, 10, 5, 
6, 11, 7, 0, 5, 1, 4, 1, 6, 8, 2, 9, 2, 8, 3, 0, 4, 6, 1])
d_dir = {}
for i, v in enumerate(dir_12):
    d_dir[i] = v 
kic_ori_seq = ['a1', 'b2', 'a1', 'd2', 'd4', 'b2', 'a1', 'b1', 'd3', 'a2', 'b3', 'a4', 'd1', 'd2', 'd1', 'a4', 'd4', 'd1', 'a1', 'c4', 'c1', 'b1', 'c1', 'a3', 'd4', 'b2', 'd1', 'd2', 'a3', 'd4', 'd3', 'b3', 'b3', 'b3', 'a3', 'a3', 'a3', 'a4', 'd1', 'd3', 'a2', 'b1', 'c3', 'a4', 'b4', 'd3', 'b4', 'c4', 'c4', 'b2', 'c2', 'b2', 'a4', 'd4', 'c4', 'b3', 'd4', 'c4', 'b2', 'b1', 'c3', 'd1', 'd4', 'b3', 'c2', 'b1', 'c4', 'a1', 'a3', 'd3', 'a4', 'd1', 'a3', 'c4', 'a2', 'a3', 'b4', 'd4', 'c3', 'c2', 'b4', 'b2', 'd4', 'c1', 'a4', 'a2', 'c4', 'b2', 'b4', 'c3', 'd3', 'c3', 'c4', 'a2', 'd3', 'a4', 'a1', 'b1', 'c2', 'd4', 'd2', 'a4', 'd3', 'a3', 'c2', 'a1', 'a2', 'a3', 'b2', 'd1', 'b4', 'a3', 'd4', 'b4', 'a2', 'a2', 'd2', 'c1', 'd1', 'b3', 'c1', 'b3', 'a3', 'd2', 'b1', 'c4', 'd1', 'b2', 'a3', 'c3', 'c2', 'a4', 'd1', 'c3', 'd3', 'b2', 'c1', 'a2', 'b4', 'c1', 'd3', 'c1', 'b2', 'c2', 'd4', 'c1', 'c1', 'b1', 'c3', 'b3', 'a1', 'b4', 'c2', 'a4', 'd2', 'a1', 'b4', 'b4', 'c3', 'b3', 'c2', 'c1', 'b4', 'b1', 'b2', 'b3', 'b3', 'b3', 'b4', 'd2', 'b3', 'c3', 'c4', 'c2', 'd2', 'd3', 'a2', 'd1', 'b2', 'a2', 'b4', 'a4', 'a1', 'c3', 'c2', 'b3', 'd2', 'b2', 'd3', 'b4', 'c1', 'd4', 'd1', 'd4', 'b1', 'c2', 'b3', 'a2', 'c3', 'c4', 'd4', 'a2', 'a1', 'b2', 'd1', 'd4', 'a2', 'a4', 'c3', 'a2', 'b3', 'a2', 'd3', 'b2', 'b1', 'a3', 'c2', 'd2', 'd3', 'c4', 'd3', 'c4', 'd4', 'b1', 'c4', 'a1', 'c1', 'a4', 'd2', 'c3', 'a1', 'c4', 'a3', 'c4', 'a1', 'd1', 'd3', 'a3', 'a1', 'c4', 'd1', 'a3', 'c3', 'c2', 'a2', 'c4', 'b1', 'd3', 'a3', 'd4', 'd2', 'd2', 'b2', 'b3', 'c1', 'c2', 'b3', 'd2', 'a3', 'c3', 'a4', 'c2', 'a1', 'a2', 'a1', 'c4', 'c1', 'c2', 'c1', 'c2', 'b1', 'a1', 'd1', 'd1', 'b4', 'd2', 'd4', 'd4', 'c4', 'd4', 'a2', 'c4', 'b1', 'b1', 'a1', 'b4', 'b3', 'b4', 'c3', 'd2', 'c4', 'c1', 'a2', 'b3', 'c3', 'a4', 'b3', 'd1', 'b1', 'b1', 'd2', 'b3', 'b2', 'a3', 'a4', 'c3', 'b1', 'a4', 'd4', 'd1', 'b1', 'd2', 'd3', 'c1', 'b1', 'a3', 'a1', 'c1', 'd1', 'b2', 'c3', 'b1', 'b3', 'c1', 'a4', 'd2', 'd4', 'b4', 'd2', 'b2', 'a4', 'd1', 'd2', 'b4', 'b4', 'd4', 'c3', 'a3', 'a4', 'd4', 'b2', 'a2', 'b2', 'c2', 'a2', 'a3', 'c2', 'b2', 'a4', 'd3', 'a4', 'b1', 'a2', 'c1', 'b2', 'd3', 'c1', 'd1', 'd3', 'd2', 'a4', 'c2', 'a1', 'c3', 'd1', 'c3', 'b1', 'c1', 'd2', 'd3', 'b4', 'a4', 'c4', 'a2', 'c2', 'd3', 'a1', 'c1', 'b1', 'c3', 'b4', 'c4', 'a2', 'a3', 'c1', 'b4', 'c2', 'c2', 'd3', 'c3', 'a3', 'a1', 'd3', 'd2', 'b3', 'a1', 'd1', 'c2', 'b4', 'a1']
d_kic_ori = {}
for i, v in enumerate(kic_ori_seq):
    d_kic_ori[i] = v 
# dir8_seq = np.array([2, 0, 7, 5, 6, 3, 3, 1, 4, 5, 4, 7, 4, 7, 5, 2, 2, 2, 2, 1, 1, 7,
#        5, 3, 1, 1, 3, 4, 2, 3, 6, 6, 0, 5, 1, 2, 7, 6, 5, 3, 5, 4, 5, 6,
#        5, 4, 4, 6, 6, 4, 0, 2, 7, 7, 0, 4, 5, 1, 2, 4, 3, 3, 4, 5, 7, 5,
#        4, 7, 2, 4, 5, 7, 0, 1, 2, 2, 3, 6, 1, 3, 5, 1, 5, 1, 2, 6, 0, 1,
#        7, 1, 0, 6, 7, 6, 5, 3, 3, 2, 5, 7, 0, 4, 3, 0, 1, 4, 1, 3, 5, 2,
#        0, 1, 6, 0, 5, 7, 1, 7, 3, 0, 6, 7, 0, 2, 5, 2, 3, 7, 4, 3, 3, 6,
#        3, 1, 2, 1, 5, 7, 6, 0, 3, 3, 0, 4, 0, 7, 1, 2, 0, 3, 0, 0, 6, 5,
#        2, 5, 4, 2, 7, 0, 1, 3, 2, 7, 6, 6, 1, 0, 1, 5, 6, 0, 3, 2, 6, 4,
#        7, 0, 7, 4, 6, 5, 4, 1, 6, 2, 4, 4, 2, 4, 7, 6, 0, 4, 3, 1, 6, 6,
#        0, 7])
# for i, v in enumerate(dir8_seq):
#     d_dir[i] = v 

## Loading .npy files

In [ ]:
matches = [] # list of experiment folders
source_folder = r"U:\Data\pak6\OpenEphys\probe_64DB\WT\illusory_contours\Abutting lines\set1\right2"
#for source_folder in source_folders:
for root, dirnames, filenames in os.walk(source_folder):
    for filename in fnmatch.filter(filenames, '*rez.mat'):
        matches.append(os.path.join(root, filename))
print matches[:5]
print matches[-5:]
print len(matches)

In [ ]:
ls = []
labels = []
for path in matches:
    path = os.path.split(path)[0]
    path_cluster_groups = os.path.join(path, 'cluster_groups.csv')
    cluster_groups = pd.read_csv(path_cluster_groups, sep = '\t')
    spike_times = np.load(os.path.join(path, 'spike_times.npy'))
    spike_clusters = np.load(os.path.join(path, 'spike_clusters.npy'))
    templates = np.load(os.path.join(path, 'templates.npy'))
    spike_templates = np.load(os.path.join(path, 'spike_templates.npy'))
    df = pd.DataFrame({'sample':spike_times.flatten(), 
                   'cluster_id':spike_clusters.flatten(), 
                   'templates':spike_templates.flatten() })
    
    for idx, i in enumerate(df.cluster_id.unique()):
        tmt_id = df[df.cluster_id==i].templates.unique().tolist()
        tmt_arr = templates[tmt_id]
        tmt_arr = np.mean(tmt_arr, axis=0)
        tmp = tmt_arr.flatten()
        ls.append(tmp)
        labels.append(cluster_groups[cluster_groups.cluster_id==i].group.values[0])
out = np.matrix(ls)   

    

In [ ]:
matches = [ r"U:\Data\pak6\OpenEphys\probe_64DA\WT\illusory contours\KIC-sqr-ori\set1\KICsqrori2-left1\ksort2",
           r"U:\Data\pak6\OpenEphys\probe_64DA\WT\illusory contours\KIC-sqr-ori\set1\KICsqrori2-right2\ksort2",
           r"U:\Data\pak6\OpenEphys\probe_64DA\WT\illusory contours\KIC-sqr-ori\set2\right\KICsqrori6-right2\ksort2",
           r"U:\Data\pak6\OpenEphys\probe_64DA\WT\illusory contours\KIC-sqr-ori\set2\left\KICsqrori6-left1\ksort2",
           r"U:\Data\pak6\OpenEphys\probe_64DA\WT\illusory contours\KIC-sqr-ori\set2\left\KICsqrori8-left1\ksort2",
]
for path in matches[:]:
    et = path.split('\\')[-2].split()[0]
    print et

## Locally sparse noise rf mapping

In [ ]:
path = r"U:\Data\pak6\OpenEphys\probe_64DA\WT\illusory contours\KIC-sqr-ori\set1\KICsqrori1-left1\ksort2"
# path = r"u:\Data\tang232\Visual_field\09.30.2017 RF+Size\openephys\10.07.2017_CC#_ET#05_post\001 4x4 RF_2017-10-07_14-09-47"
# path = r"U:\Data\tang232\Receptive_field\4X4 RF Mapping+ Size\07.01.2017_CC#_ET#400_pre\conc"
path_cluster_groups = os.path.join(path, 'cluster_groups.csv')
cluster_groups = pd.read_csv(path_cluster_groups, sep = '\t')
# good_units = cluster_groups[cluster_groups.group != 'noise'].cluster_id.values
noise_units = cluster_groups[(cluster_groups['group'] == 'noise') | (cluster_groups['group'] == 'mua')].cluster_id.values


In [ ]:
for fname in matches[:]:
    et = fname.split('\\')[-2].split()[0]
    rec = fname.split('\\')[-2].split()[1]
    print et, rec

In [ ]:
#Receptive field mapping
# d_rf = {}
# d_rf['on'] = {}
# d_rf['off'] = {}

ls_psth = []
ls_spikes = []
for path in matches[:]:
    path = r"U:\Data\pak6\OpenEphys\probe_64DA\WT\illusory contours\KIC-ori\right\KICori2-right1\conc"
    et = path.split('\\')[-2].split()[0]
    
    path_cluster_groups = os.path.join(path, 'cluster_groups.csv')
    cluster_groups = pd.read_csv(path_cluster_groups, sep = '\t')
    noise_units = cluster_groups[(cluster_groups['group'] == 'noise') | 
                                 (cluster_groups['group'] == 'mua')].cluster_id.values
    spike_times = np.load(os.path.join(path, 'spike_times.npy'))
    spike_clusters = np.load(os.path.join(path, 'spike_clusters.npy'))
    templates = np.load(os.path.join(path, 'templates.npy'))
    spike_templates = np.load(os.path.join(path, 'spike_templates.npy'))
    df = pd.DataFrame({'times':spike_times.flatten()/30000.0, 
                       'cluster_id':spike_clusters.flatten(), 
                       'templates':spike_templates.flatten() })

    df = df[~df.cluster_id.isin(noise_units)]
    df_rez = df
#     break
#     df_rez['trial_n'] = df_rez.times//4
    
    df2 = df[df.times <= 20]
    
    df3 = df[(df.times > 20)   & (df.times<= 220)]
    df3.times = df3.times - 20

    df4 = df[ (df.times > 220) & (df.times <= 1000)]
    df4.times = df4.times - 220
    
    df5 = df[ (df.times > 1000)]
    df5.times = df5.times - 1000
    break

#     df6 = df[df.times > 1000 ]
#     df6.times = df6.times - 1000

# #     df2['trial_n'] = df2.times//4.0
# #     df3['trial_n'] = df3.times//60.0
#     df5['trial_n'] = df5.times//2.0
#     df6['trial_n'] = df6.times//2.0
    
#     df5['stim1'] = df5['trial_n'].map(d_stim)
#     df6['stim1'] = df6['trial_n'].map(d_stim)
    
#     df_g1 = df_rez
#     df_g1['et'] = '476'
#     df_g1['rec'] = 'left-1'
#     df_g1['fname'] = path
    df5['trial_n'] = df5.times//2.0
    df5['stim1'] = df5['trial_n'].map(d_kic_ori)
    df5['trial_spikes'] = df5.times - df5['trial_n']*2
    tmp_psth0, _ ,_ = g1_psth(df5)
    break
#     tmp_psth0['et'] = et
#     tmp_psth0['cluster_id'] = tmp_psth0['cluster_id'].astype('str') + "-" + et
#     tmp_psth0['fname'] = path
# #     ls_spikes.append(tmp_spikes0)
#     ls_psth.append(tmp_psth0)
    
#     tmp_psth1, _, _ = g1_psth(df6)
#     ls_spikes.append(tmp_spikes1)
#     ls_psth.append(tmp_psth1)
    
#     df_rf = df3
#     d_rf = avg_rf_on_off(df_rf, d_rf , et, rec)
    
#     df_kic = df4
#     df_kic2 = df5
# g1_ms_psth_v1 = pd.concat(ls_psth)
# g1_ms_spikes_v1 = pd.concat(ls_spikes)

In [ ]:
kic_sqr_ori_psth.to_pickle('kic_sqr_ori_psth.pkl')

## PSTH for Direction Tuning

In [ ]:
def dir_tuning(data, et, angle_step, paradigm, d_dir):
    
    data['trial_n'] = data.times//1.0
    data['stim1'] = data['trial_n'].map(d_dir)
    data['trial_spikes'] = data.times - data['trial_n']*1
    data['angle'] = data.stim1*30

    for unit in data['cluster_id'].unique():
        tmp = data[data.cluster_id == unit]
        for idx, val in enumerate(sorted(tmp.stim1.unique())):
            tmp2 = tmp[tmp.stim1 == val]
            df = ena.getRaster_kilosort(tmp2, unit, trial_length) 
    #         if len(df.times) < 100:
    #             continue
            cluster_id = str(unit) + '-' + str(et) 
            cuid = str(unit) + str('et') + str(et) +  str(val)
            h, ttr = ena.PSTH(df.times, th_bin, trial_length, trials_number) # all times rescaled to 0-4 this is why trias number 1.0
            zscore = sstat.mstats.zscore(h)
            mean = np.mean(h[:30])
            std = np.std(h[:30])
            if mean==0:
                std=1
            ztc = (h - mean)/std

            df_psth_tmp = pd.DataFrame({     'times':ttr,    'Hz':h,  'stim1': val , 'paradigm': paradigm,
                                  'et': et,   'cluster_id':cluster_id, 'abs_times': ttr + val*1, 'angle': val*angle_step,
                                    'zscore':zscore, 'ztc':ztc, 'cuid':cuid  })
            ls_psth.append(df_psth_tmp)

    df_tuning = pd.concat(ls_psth)
    return df_tuning

In [ ]:
sorted(matches)

In [ ]:
trials_number = 15.0
trial_length = 1.0
th_bin = 0.01
ls_psth = []
ls_tuning = []

for path in sorted(matches)[1:]:
    path = os.path.split(path)[0]

    et = path.split('\\')[-2].split()[0]
    
    path_cluster_groups = os.path.join(path, 'cluster_groups.csv')
    cluster_groups = pd.read_csv(path_cluster_groups, sep = '\t')
    noise_units = cluster_groups[(cluster_groups['group'] == 'noise') | 
                                 (cluster_groups['group'] == 'mua')].cluster_id.values
    spike_times = np.load(os.path.join(path, 'spike_times.npy'))
    spike_clusters = np.load(os.path.join(path, 'spike_clusters.npy'))
    templates = np.load(os.path.join(path, 'templates.npy'))
    spike_templates = np.load(os.path.join(path, 'spike_templates.npy'))
    df = pd.DataFrame({'times':spike_times.flatten()/30000.0, 
                       'cluster_id':spike_clusters.flatten(), 
                       'templates':spike_templates.flatten() })

    df = df[~df.cluster_id.isin(noise_units)]

    df1 = df[ (df.times > 20) & (df.times <= 200)]
    df1.times = df1.times - 20
    tmp_tun = dir_tuning(df1, et ,45, 'lg1', d_dir)
    
    df2 = df[ (df.times > 200) & (df.times <= 380)]
    df2.times = df2.times - 200
    tmp_tun2 = dir_tuning(df2, et ,45, 'lg2', d_dir)
    
    df3 = df[ (df.times > 380) & (df.times <= 560)]
    df3.times = df3.times - 380
    tmp_tun3 = dir_tuning(df3, et, 45, 'lg3', d_dir)
    
    
    df4 = df[ (df.times > 560) & (df.times <= 740)]
    df4.times = df4.times - 560
    tmp_tun4 = dir_tuning(df4, et ,30, 'ic1', d_dir)
    
    df5 = df[ (df.times > 740) & (df.times <= 920)]
    df5.times = df5.times - 740
    tmp_tun5 = dir_tuning(df5, et, 30, 'ic2', d_dir)
    
    df6 = df[ (df.times > 920) & (df.times <= 1100)]
    df6.times = df6.times - 920
    tmp_tun6 = dir_tuning(df6, et, 30, 'ic3', d_dir)
    
    df_tun = pd.concat([ tmp_tun, tmp_tun2, tmp_tun3, tmp_tun4, tmp_tun5, tmp_tun6 ])
    ls_psth.append(df_tun)

master_tun = pd.concat(ls_psth)
master_tun = master_tun.dropna()
master_tun['ori'] = master_tun['angle'] % 180

In [ ]:
path = r"U:\Data_Analysis\pak6\Analysis of units\Kanizsa paper ephys\kic_sqr_ori_tuning.pkl"
kic_sqr_ori_tuning = pd.read_pickle(path)
kic_sqr_ori_tuning.head()

In [ ]:
_inp2.paradigm.unique()

In [ ]:
master_tun['cuid'] = master_tun.cluster_id.astype('str') + master_tun.paradigm
_inp = master_tun[(master_tun.times > 0.35) & (master_tun.times < 0.8)]
_inp2 = _inp.groupby(['paradigm', 'cluster_id', 'cuid' , 'ori']).mean().reset_index()
base = master_tun[(master_tun.times > 0.05) & (master_tun.times < 0.35)].groupby(['paradigm', 
                        'cluster_id', 'cuid' , 'ori']).Hz.mean().values
# _inp2['bs_fr'] = _inp2.Hz - base
df_pref = _inp2.sort_values('Hz', ascending=False).groupby(['cuid'], as_index=False).first()
pref_ori = dict(zip(df_pref.cuid, df_pref.ori))
_inp2['pref_ori'] = _inp2.cuid.map(pref_ori)
# _inp2 = _inp2.groupby('cluster_id').apply(get_osi)

In [ ]:
A = np.random.randn(4,3)
B = np.sum(A, axis = 1, keepdims = True)
B

In [ ]:
# plot single unit responses
_inp = df_tuning[(df_tuning.times > 0.35) & (df_tuning.times < 0.8)]
_inp = _inp.groupby(['cluster_id', 'angle']).mean().reset_index()
sem = _inp.groupby(['cluster_id', 'angle']).sem().reset_index()
for unit in sorted(_inp.cluster_id.unique())[10:20]:
    tmp = _inp[_inp.cluster_id == unit]
    sem2 = sem[sem.cluster_id == unit]
    
    tmp2 = tmp.append(tmp.iloc[0])
    sem3 = sem2.append(sem2.iloc[0])
    ax = plt.subplot(111, projection='polar')
    
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.plot((tmp2.angle/360)*2*np.pi, abs(tmp2.Hz), label = unit, color = 'k')
    tmp2['a'] = tmp2.Hz + sem3.Hz
    tmp2['b'] = tmp2.Hz - sem3.Hz
    ax.fill_between((tmp2.angle/360)*2*np.pi, tmp2.a, tmp2.b, alpha=0.2, color = 'gray')
    ax.set_xticks(np.arange(0,2*np.pi,np.pi/4))
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
#     plt.savefig('polar_dir_tuning-261-kicsqrori2-left1.pdf')

In [ ]:
sns.distplot(_inp2.groupby('cluster_id').osi.mean().values)

In [ ]:
# compute osi
def get_osi(data):
    tmp = data[data.paradigm == 'lg']
    pref_ori = tmp.pref_ori.unique()[0]
    orth_ori = (pref_ori + 90)%180
    pref_fr = tmp[tmp.ori == pref_ori].Hz.values
    orth_fr = tmp[tmp.ori == orth_ori].Hz.values
    osi = (pref_fr - orth_fr)/(pref_fr + orth_fr)
    data['osi'] = osi[0]
    return data
    

In [ ]:
# _inp = df_tun[(df_tun.times > 0.35) & (df_tun.times < 0.6)]
# _inp = _inp.groupby(['paradigm', 'ori' ,'cluster_id']).mean().reset_index()
# sem = _inp.groupby(['cluster_id', 'angle']).sem().reset_index()
for unit in sorted(_inp2.cluster_id.unique())[:]:
    tmp = _inp2[_inp2.cluster_id == unit]
    tmp2 = tmp[tmp.paradigm == 'lg3']
    tmp3 = tmp[tmp.paradigm == 'ic3']
#     tmp4 = tmp[tmp.paradigm == 'lg3']
#     osi = tmp2.osi.values[0]
    f, ax = plt.subplots()
    x1 = tmp2.ori.unique()
    x2 = tmp3.ori.unique()
    x3 = tmp4.ori.unique()
    
    ax.plot( x1, tmp2.Hz, label = tmp2.pref_ori.values[0], color = 'k')
    ax.plot( x2, tmp3.Hz, label = tmp3.pref_ori.values[0], color = 'red')
#     ax.plot( x3, tmp4.Hz, label = tmp4.pref_ori.values[0], color = 'blue')

    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
#     plt.savefig('polar_dir_tuning-261-kicsqrori2-left1.pdf')

In [ ]:
high_fr_units = _inp.groupby('cluster_id').Hz.mean().reset_index()
high_fr_units = high_fr_units[high_fr_units.Hz < 5].cluster_id.unique()

In [ ]:
# _inp = _inp[_inp.cluster_id.isin(high_fr_units)]
# _inp = _inp[_inp.layer == 'l2_3']
idx = _inp.groupby('cluster_id').Hz.transform(max) == _inp['Hz']
grat_pref = dict(zip(_inp[idx].cluster_id, (_inp[idx].angle) ))
grat_opp = {} 
for k in grat_pref.keys():
    pref = grat_pref[k]
    if pref >= 180:
        opp_dir = pref - 180
    else:
        opp_dir = pref + 180
    grat_opp[k] = opp_dir
df_tun['pref_dir'] = df_tun.cluster_id.map(grat_pref)
df_tun['opp_dir'] = df_tun.cluster_id.map(grat_opp)

In [ ]:
def g1_psth(data):
    ls = []
    ls_tmt = []
    ls_spikes = []
#     exp  = fname.split('_')[0]
#     data['opto'] = data.trial_n.map(d_opto)
    data['opto'] = 0
#     data['stim1'] = data.trial_n.map(d_stim)
    data['trial_spikes'] = data.times - data.trial_n*2
    for i, v in enumerate(sorted(data.opto.unique())): 
        tmp = data[data['opto'] == v ]
        opto_time = 0
        if v == 1:
            opto_time = 8
        for idx, val in enumerate(sorted(tmp.stim1.unique())): 
            tmp2 = tmp[tmp['stim1'] == val ]
            
            for unit in tmp2['cluster_id'].unique():
                df = ena.getRaster_kilosort(tmp2, unit, 2.0) 
                if len(df.times) < 10:
                    continue
        
#                 cluster_id = str(unit) + str(exp) + 'et' + str(et) + str(rec) 
#                 cuid =  str(cluster_id) + 'stim' + str(val) + 'opto' + str(v)
                
#                 tmt, depth, ch_idx = ena.ksort_get_tmt(tmp2, unit, templates, channel_groups)
#                 df_tmt_tmp = pd.DataFrame({'tmt':tmt,  'cluster_id' : cluster_id, 'path': path })
#                 ls_tmt.append(df_tmt_tmp)
#                 tmp3 = tmp2[tmp2.cluster_id == unit]
#                 tmp3['cluster_id'] = cluster_id
#                 tmp3['stim1'] = str(val) + str(v)
#                 ls_spikes.append(tmp3)
                
                #df = tmp[tmp.cluster_id==unit]
                #df.trial_st = df.trial_st-t_idx[0]
     
                h, ttr = ena.PSTH(df.times, 0.01, 2.0, 25.0) # all times rescaled to 0-4 this is why trias number 1.0
                zscore = sstat.mstats.zscore(h)
                mean = np.mean(h[:50])
                std = np.std(h[:50])
                if mean==0:
                    std=1
                ztc = (h - mean)/std

                df_psth = pd.DataFrame({     'times':ttr,    'Hz':h, 
                                            'stim1': str(val) + str(v),  'opto':v, 'abs_times': idx*2.0 + ttr + opto_time,
                                        'zscore':zscore, 'ztc':ztc,   'cluster_id':unit,  })
                ls.append(df_psth)
            

    
    df_psth = pd.concat(ls)
  
    df_spikes = 0
    df_tmt = 0
    return df_psth, df_spikes, df_tmt

In [ ]:
def extract_probe_idx(fname):
    searchObj=re.search(r'probes_[\d]+', fname)
    return int(searchObj.group()[7:])

### Plot neural responses

In [ ]:
# %matplotlib notebook
f, ax = plt.subplots(figsize = (8, 4), facecolor = 'w')
tmp = test[test.fname.isin(good_recs)].dropna()
data = tmp[(tmp.kic_sig == False)]
# data = data[data.cluster_id.isin(data2.cluster_id.unique())]
# data = data[data.et.str.startswith('7')]
# data = data[data.depth < 500]
# data = data[data.cluster_id.isin(result[result.n_type == 'un'].cluster_id.unique())]
data = data[(data.stim1.str.contains('0'))]
data = data.dropna()
# data = data[(data.stim1.str.startswith('1')) | (data.stim1.str.startswith('2')) ]
# data = data[ (data.times >= 0.5) & (data.times < 3)]
# data = data.sort_values(by = ['kic_ind'])
hm = data.pivot('cluster_id', 'abs_times', 'zscore')
hm = hm.dropna()
hm2 = hm.values[ np.argsort(np.mean(hm.values[:,100:150], axis = 1) - np.mean(hm.values[:,50:100], axis = 1) )]

sns.heatmap(hm2, cmap = 'jet',  annot=False, xticklabels=  200, vmax = 7, cbar = False,
            vmin = -1, robust = True, yticklabels=False, ax = ax )
ax.set(xlabel='', ylabel= hm.shape[0])
# plt.savefig('hm_g1_lm_opto_n2.png')

In [ ]:
f, ax = plt.subplots(figsize = (8, 4), facecolor = 'w')
colors = ['k', 'g']
sns.tsplot(data.reset_index(), time = 'times', value = zsc,  ci = 68,
   unit = 'cluster_id', condition = 'stim1', legend = False,   estimator=np.nanmean)
sns.despine()
plt.axvspan(1, 1.5, alpha=0.2, color='gray')
plt.axvspan(0.9, 1.7,ymin = 0.95, ymax = 1, alpha=0.5, color='green')
# plt.savefig('line_g1_lm_n2_opto.pdf')

In [ ]:
# open .gif files using pil
fname = r"U:\Visual Stimulation\pak6\Vis Stim\RF mapping\probes1.gif"
from PIL import Image
frames_ls = []
im = Image.open(fname)
# skip to the second frame
tmp = im.convert('L')
frame_arr = np.array(tmp)
frames_ls.append(frame_arr)

try:
    while 1:
        im.seek(im.tell()+1)
        tmp = im.convert('L')
        frame_arr = np.array(tmp)
        frames_ls.append(frame_arr)
        #plt.imshow(im, cmap = plt.cm.gray, vmin=0, vmax=255)
        
except EOFError:
    pass # end of sequence
frames_arr = np.stack(frames_ls, axis = 0)

In [ ]:
data.groupby(['rec', 'et', 'exp']).mean()

In [ ]:
import glob
matches = [] # list of experiment folders
source_folder = r"U:\Visual Stimulation\pak6\Vis Stim\RF mapping\test_probes_on"
#for source_folder in source_folders:
file1 = glob.glob(source_folder + '\\probes_*.bmp')
file_list=sorted(file1, key=lambda x: extract_probe_idx(x))


In [ ]:
from PIL import Image
frames_ls = []
for fname in file_list[:]:
    img = Image.open(fname).convert('L')
    tmp = np.array(img)
    frames_ls.append(tmp)

#     plt.imshow(img, cmap = plt.cm.gray, vmin=0, vmax=255)
#     plt.show()
frames_arr = np.stack(frames_ls, axis = 0)

In [ ]:
plt.imshow(np.mean(frames_arr, axis = 0), cmap = plt.cm.gray)
plt.colorbar()

In [ ]:
path = r"U:\Visual Stimulation\pak6\Vis Stim\RF mapping\probes_off.npy"
frames_arr_off = np.load(path)
path = r"U:\Visual Stimulation\pak6\Vis Stim\RF mapping\probes_on.npy"
frames_arr_on = np.load(path)

In [ ]:
# extract frame indices from frames
# plot single STA frames
fig = plt.subplots(facecolor='white')
d_frames_idx = {}
df_rez = df4
for unit in sorted(df_rez.cluster_id.unique())[80:90]:
    tmp = df_rez[df_rez.cluster_id == unit]
    # account for latency and 120 empty frames, convert to frames idx
    tmp =  np.rint((tmp.times.values[:] + 0.05 -2)*60)
    
#     frame_idx = [round_to_frame(x) for x in tmp]
    frame_idx = ((tmp//15))
    frame_idx = frame_idx[frame_idx>-1]
    frame_idx = frame_idx[frame_idx < 3002]
    frame_idx = frame_idx.astype(int)
    
    unique, counts = np.unique(frame_idx, return_counts=True)
    d_frames_idx = dict(zip(unique, counts))
    avg_frame_on = np.average(frames_arr_on[d_frames_idx.keys()],axis = 0, weights=d_frames_idx.values())
    avg_frame_off = np.average(frames_arr_off[d_frames_idx.keys()],axis = 0, weights=d_frames_idx.values())
    print unit
    plt.imshow(avg_frame_on, cmap = plt.cm.gray)
    plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    plt.tick_params(axis='y', which='both', right=False, left=False, labelleft=False)
    for pos in ['right','top','bottom','left']:
        plt.gca().spines[pos].set_visible(False)
    plt.show()
#     plt.savefig('rf_map_on-12-kicsqr2-right1.pdf')
    plt.imshow(avg_frame_off, cmap = plt.cm.gray)
    plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    plt.tick_params(axis='y', which='both', right=False, left=False, labelleft=False)
    for pos in ['right','top','bottom','left']:
        plt.gca().spines[pos].set_visible(False)
    plt.show()
# plt.savefig('rf_map_off-12-kicori2-right1.pdf')

In [ ]:

for unit in data.cluster_id.unique():
    print unit
    try:
        plt.imshow(d_rf['on'][unit], cmap = plt.cm.gray)
        plt.colorbar()
        plt.show()
        plt.imshow(d_rf['off'][unit], cmap = plt.cm.gray)
        plt.colorbar()
        plt.show()
    except:
        continue

In [ ]:
np.arange(10)

In [ ]:
stim_type = sorted(data.stim1.unique())
colors = my_pal[:]
psth_tmp = data[data.cluster_id == '199005et715left']
psth_tmp = psth_tmp[(psth_tmp.stim1.str.startswith('1')) | (psth_tmp.stim1.str.startswith('2'))]
for unit in sorted(psth_tmp.cluster_id.unique())[:]:
#     print unit
    f, ax = plt.subplots(1 , figsize = (10,4), sharey=True, sharex=True, facecolor = 'w')
    tmp = psth_tmp[psth_tmp.cluster_id == unit]
    sns.despine()
    plt.suptitle(unit)
    for idx, stim in enumerate(sorted(tmp.stim1.unique())):
        ax.plot(tmp[(tmp.stim1 == stim)].Hz, label = stim_type[idx], color = colors[idx])
#         plt.xlim(100,200)
    
#     for idx, stim in enumerate(sorted(tmp[tmp.opto == 1].stim1.unique())):
#         ax[1].plot(tmp[(tmp.stim1 == stim)].Hz, label = stim_type[idx])
# #         plt.xlim(100,200)
#     plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#     plt.show()
plt.axvspan(100, 150, alpha = 0.1, color='k')
plt.axvspan(50, 100, alpha = 0.05, color='k')
sns.despine()
# plt.savefig('line_199005et715left_opto_12.pdf')

In [ ]:
ic_mod_units = ['10-KICsqrori6-right2', '11-KICsqrori2-right2', '171-KICsqrori2-left1', '176-KICsqrori6-right2',
               '207-KICsqrori2-right2', '208-KICsqrori2-right2', '214-KICsqrori8-left1', '217-KICsqrori2-right2',  
                '232-KICsqrori6-right2', '257-KICsqrori6-left1', '261-KICsqrori2-left1', '31-KICsqrori8-left1',
                '70-KICsqrori6-right2', '79-KICsqrori2-right2', '86-KICsqrori2-left1'
               ]
ori_ls = ['a', 'b', 'c', 'd']
colors = my_pal[::2]
tmp_psth0 = kic_sqr_ori_psth[kic_sqr_ori_psth.cluster_id.isin(ic_mod_units)]
stim_type = sorted(kic_sqr_ori_psth.stim1.unique())
for unit in sorted(tmp_psth0.cluster_id.unique())[74:75]:
#     print unit
#     unit = '261-KICsqrori2-left1'
    f, ax = plt.subplots(4, 1 , figsize = (6,8), sharey=True, sharex=True, facecolor = 'w')
    plt.suptitle(unit)
    tmp = tmp_psth0[tmp_psth0.cluster_id == unit]
    sns.despine()
    for plot_idx, ori in enumerate(sorted(ori_ls)):
        tmp2 = tmp[tmp.stim1.str.contains(ori)]
        for idx, stim1 in enumerate(sorted(tmp2.stim1.unique())[:]):
            ax[plot_idx].plot(tmp2[tmp2.stim1 == stim1].Hz, label = stim_type[idx], color = colors[idx] )
       
        for i in range(2):
            ax[plot_idx].axvspan(50+50*i, 100+50*i, alpha=0.1, color='k')    
#     plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#     plt.show()
# plt.axvspan(100, 150, alpha = 0.1, color='k')
# plt.axvspan(50, 100, alpha = 0.05, color='k')

# plt.savefig('line_119-KICori2-right1.pdf')

In [ ]:
_inp = df_tuning[(df_tuning.times > 0.35) & (df_tuning.times < 0.85)]
_inp['ori'] = _inp.angle%180
_inp = _inp.groupby(['cluster_id', 'ori']).mean().reset_index()



In [ ]:
_inp = tmp_psth0
arr = _inp.stim1.values
ori = [x[0] for x in arr]
stim = [x[1] for x in arr]
_inp['ori'] = ori
_inp['stim'] = stim
ind_fr = _inp[(_inp.times > 0.55) & (_inp.times < 1.05)].groupby(['stim' ,'ori', 'cluster_id']).mean().reset_index().Hz
ic_df = _inp[(_inp.times > 1.05) & (_inp.times < 1.5)].groupby(['stim', 'ori' , 'cluster_id']).mean().reset_index()
ic_df['im_cir'] = (ic_df.Hz - ind_fr)/(ic_df.Hz + ind_fr)
ori_2deg = {'a':0, 'b':1, 'c':2, 'd':3}
ic_df['deg'] = ic_df['ori'].map(ori_2deg)

ic_df = ic_df[ic_df.stim == '1']
idx = ic_df.groupby('cluster_id')['im_cir'].transform(max) == ic_df['im_cir']
ic_pref = dict(zip(ic_df[idx].cluster_id, (ic_df[idx].deg) ))
ic_df['ic_pref'] = ic_df.cluster_id.map(ic_pref)
ic_df['rel_deg'] = ic_df['deg'].astype(int) - ic_df['ic_pref'].astype(int)


In [ ]:
_inp['norm'] = _inp.groupby('cluster_id')['Hz'].apply(lambda x: (x-x.min())/(x.max()-x.min()))
_inp = _inp[_inp.cluster_id == '119-KICori2-right1']
# ic_df = ic_df[ic_df.im_cir > 0]
sns.factorplot(x = 'ori', y = 'norm', data = _inp, height = 4, 
               aspect = 1.5, color = 'gray')
# plt.ylim(0, 1)
# plt.savefig('point_119-KICori2-right1-grating-tuning.pdf')


In [ ]:
grat_tun = kic_sqr_ori_tuning[kic_sqr_ori_tuning.cluster_id.isin(ic_mod_units)]
grat_tun = grat_tun[(grat_tun.times > 0.35) & (grat_tun.times < 0.8)]
grat_tun['ori'] = grat_tun.angle%180
_inp = grat_tun.groupby(['cluster_id', 'ori']).Hz.mean().reset_index()

In [ ]:
idx = _inp.groupby('cluster_id').Hz.transform(max) == _inp['Hz']
grat_pref = dict(zip(_inp[idx].cluster_id, (_inp[idx].ori) ))
_inp['grat_pref'] = _inp.cluster_id.map(grat_pref)
_inp['rel_deg'] = _inp['ori'].astype(int) - _inp['grat_pref'].astype(int)

In [ ]:
grat_tun.ori.unique()

In [ ]:
ls = []
inp_stat = ic_df
for i in sorted(inp_stat.rel_deg.unique()):
    tmp = inp_stat[inp_stat.rel_deg == i].im_cir.values
    ls.append(tmp)
    print np.mean(tmp), sstat.sem(tmp)
    print len(tmp)
sstat.kruskal(ls[0], ls[1], ls[2], ls[3], ls[4], ls[5])

In [ ]:
df1 = ic_df[ic_df.deg == '0'].groupby('grat_pref').cluster_id.count().reset_index()
df2 = ic_df[ic_df.deg == '0'].groupby('ic_pref').cluster_id.count().reset_index()
df1 = pd.melt(df1, id_vars = ['cluster_id'], value_vars = ['grat_pref'])
df2 = pd.melt(df2, id_vars = ['cluster_id'], value_vars = ['ic_pref'])
df3 = pd.concat([df1, df2])
sns.factorplot(x = 'value', y = 'cluster_id', hue = 'variable', data = df3, 
               kind = 'bar', palette = ['gray', 'crimson'], height = 4, aspect = 1, legend = False)
# plt.savefig('bar_counts_pref-ori.pdf')

In [ ]:
arr = tmp.stim1.values
ori = [x[0] for x in arr]
stim = [x[1] for x in arr]
tmp['ori'] = ori
tmp['stim'] = stim

In [ ]:
# tmp2 = tmp[tmp.stim1.str.contains('1')]
fig_inp_stim = tmp[(tmp.times > 1.05) & (tmp.times < 1.5)].groupby(['stim', 'ori']).mean().reset_index()
fig_inp_ind = tmp[(tmp.times > 0.55) & (tmp.times < 1.05)].groupby(['stim', 'ori']).mean().reset_index().Hz
fig_inp['im_cir'] = (fig_inp_stim.Hz - fig_inp_ind)/(fig_inp_stim.Hz + fig_inp_ind)
sns.factorplot(x = 'ori', y = 'im_cir', data = fig_inp, palette = colors,
               height = 4, aspect = 1.5, hue = 'stim',)
# plt.savefig('point_ic-tuning_119-KICori2-right1.pdf')

In [ ]:
df5['trial_n'] = df5.times//2.0
df5['stim1'] = df5['trial_n'].map(d_kic_ori)
df5['trial_spikes'] = df5.times - df5['trial_n']*2


In [ ]:
colors = my_pal[:]
colors = colors[::2]
ori_ls = ['a', 'b', 'c', 'd']

for unit in sorted(df5.cluster_id.unique())[84:85]:
    tmp = df5[df5.cluster_id == unit]
    f, ax = plt.subplots( 4, 1, figsize = (8, 9),  sharex=True, sharey=True, facecolor = 'w')
    plt.suptitle(unit)
    for plot_idx, ori in enumerate(ori_ls):    
#         print ori
        tmp2 = tmp[tmp.stim1.str.contains(ori)]
        stim_type = sorted(tmp2.stim1.unique())
        for idx, stim in enumerate(sorted(tmp2.stim1.unique()[:])):
            un_trials = np.unique(tmp2[(tmp2.stim1 == stim)].trial_n.values, return_counts=True)[0].size
            counts = np.unique(tmp2[(tmp2.stim1 == stim)].trial_n.values, return_counts=True)[1]
            y = np.arange(1, (un_trials+1))
            y = np.repeat(y, counts)
            x = tmp2[(tmp2.stim1 == stim)].trial_spikes + idx*2
            ax[plot_idx].plot(x, y, '.', label = stim_type[idx], color = colors[idx], markersize = 6 )
        #         plt.xlim(100,200)
        # plt.gca().invert_yaxis()
    #     for idx, stim in enumerate(sorted(tmp[tmp.opto == 1].stim1.unique()[:])):
    #         un_trials = np.unique(tmp[(tmp.stim1 == stim)].trial_n.values, return_counts=True)[0].size
    #         counts = np.unique(tmp[(tmp.stim1 == stim)].trial_n.values, return_counts=True)[1]
    #         y = np.arange(1, (un_trials+1))
    #         y = np.repeat(y, counts)
    #         x = tmp[(tmp.stim1 == stim)].trial_spikes + idx*2
    #         ax[1].plot(x, y, '.' ,  label = stim_type[idx], color = colors[idx])
        #         plt.xlim(100,200)
#     plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    
        for i in range(4):
            ax[plot_idx].axvspan(1+2*i, 1.5+2*i, alpha=0.2, color='k')
#         ax[i].axvspan(0.5 + 2*i, 1 + 2*i, alpha=0.2, color='gray')
    sns.despine()
    
#     plt.savefig('raster_119-kicori2-right1.png')
#     plt.show()

In [ ]:
colors = my_pal[4:]
# colors = colors[::2]
sp_tmp = df_spikes[df_spikes.cluster_id == '199005et715left']
sp_tmp = sp_tmp[(sp_tmp.stim1.str.startswith('3')) | (sp_tmp.stim1.str.startswith('4'))]
for unit in sorted(sp_tmp.cluster_id.unique())[:]:
    tmp2 = sp_tmp[sp_tmp.cluster_id == unit]
    f, ax = plt.subplots(  1, figsize = (10, 4),  sharex=True, sharey=True, facecolor = 'w')
    plt.suptitle(unit)

    stim_type = sorted(tmp2.stim1.unique())
    for idx, stim in enumerate(sorted(tmp2.stim1.unique()[:])):
        un_trials = np.unique(tmp2[(tmp2.stim1 == stim)].trial_n.values, return_counts=True)[0].size
        counts = np.unique(tmp2[(tmp2.stim1 == stim)].trial_n.values, return_counts=True)[1]
        y = np.arange(1, (un_trials+1))
        y = np.repeat(y, counts)
        x = tmp2[(tmp2.stim1 == stim)].trial_spikes + idx*2
        ax.plot(x, y, '.', label = stim_type[idx], color = colors[idx] )

    for i in range(4):
        ax.axvspan(1+2*i, 1.5+2*i, alpha=0.2, color='k')
#         ax[i].axvspan(0.5 + 2*i, 1 + 2*i, alpha=0.2, color='gray')
    sns.despine()
    
# plt.savefig('raster_199005et715left_opto_34.pdf')
#     plt.show()

In [ ]:
# tmp_psth0['et'] = 'KIC13'
# tmp_psth0['exp'] = '004'
# tmp_psth0['cluster_id'] = '004' + tmp_psth0['cluster_id'].astype('str') + 'KIC13' 
# tmp_psth1['et'] = 'KIC13'
# tmp_psth1['exp'] = '005'
# tmp_psth1['cluster_id'] = '005' + tmp_psth1['cluster_id'].astype('str') + 'KIC13' 
kic_psth = pd.concat([kic_psth0, kic_psth1])

# Kanizsa SC analysis

In [ ]:
matches = [] # list of experiment folders
source_folder = r"C:\Users\Chub_lab\Desktop\phy\rf_tmp\conc"
#for source_folder in source_folders:
for root, dirnames, filenames in os.walk(source_folder):
    for filename in fnmatch.filter(filenames, '*rez.mat'):
        matches.append(os.path.join(root, filename))

print matches[:5]
print matches[-5:]
print len(matches)

In [ ]:
matches = [] # list of experiment folders
source_folder = r"U:\Data\pak6\OpenEphys\probe_64DB\WT\illusory_contours\LM-Arch"
for root, dirnames, filenames in os.walk(source_folder):
    for dirname in fnmatch.filter(dirnames, '*conc'):
        if 'phy' in dirname or 'bad' in dirname:
            continue
        matches.append(os.path.join(root, dirname)) 
print matches[:5]
print len(matches)

In [ ]:
files = [f for f in matches if '201' in f or 'KIC' in f]
files

In [ ]:
for path in matches[:]:
    
    et = path.split('\\')[-2].split()[0]
    rec = path.split('\\')[-2].split()[1]
    stims = sorted([x for x in  get_immediate_subdirectories(os.path.split(path)[0]) if '201' in x])
    print et, rec, stims[4]
    for fname in stims[:]:

        if 'kic-opto' in fname:

            exp  = fname.split('_')[0]
            stim1 = fname.split('_')[1]
        else:
            continue
       
 

In [ ]:
len(trial_seq)

In [ ]:

# f, axs = plt.subplots(3,2, sharex=True, sharey = True, figsize =(14,10))
data = df_rez
trial_length = 4
plt.figure()
for i in sorted(data['cluster_id'].unique()[:]):
    print i
    df_r = ena.getRaster_kilosort(data, i, trial_length)
    
    plt.plot( df_r.times, df_r.trial, '.')
    plt.gca().invert_yaxis()
    plt.xlim(0.5,2)
    plt.show()
sns.despine()

# plt.ylabel('Trials')
# plt.tight_layout()
# plt.savefig('rasrter2_123.pdf')

In [ ]:
probe = '64DB'
channel_groups = ena.get_channel_depth(probe)
def make_kic_psth(data, fname, et, rec, d_stim, d_opto, templates, channel_groups ):
    ls = []
    ls_tmt = []
    ls_spikes = []
    exp  = fname.split('_')[0]
#     data['opto'] = data.trial_n.map(d_opto)
    data['opto'] = 0
    data['stim1'] = data.trial_n.map(d_stim)
    data['trial_spikes'] = data.times - data.trial_n*2
    for i, v in enumerate(sorted(data.opto.unique())): 
        tmp = data[data['opto'] == v ]
        opto_time = 0
        if v == 1:
            opto_time = 8
        for idx, val in enumerate(sorted(tmp.stim1.unique())): 
            tmp2 = tmp[tmp['stim1'] == val ]
            
            for unit in tmp2['cluster_id'].unique():
                df = ena.getRaster_kilosort(tmp2, unit, 2.0) 
                if len(df.times) < 100:
                    continue
        
                cluster_id = str(unit) + str(exp) + 'et' + str(et) + str(rec) 
                cuid =  str(cluster_id) + 'stim' + str(val) + 'opto' + str(v)
                
                tmt, depth, ch_idx = ena.ksort_get_tmt(tmp2, unit, templates, channel_groups)
#                 df_tmt_tmp = pd.DataFrame({'tmt':tmt,  'cluster_id' : cluster_id, 'path': path })
#                 ls_tmt.append(df_tmt_tmp)
#                 tmp3 = tmp2[tmp2.cluster_id == unit]
#                 tmp3['cluster_id'] = cluster_id
#                 tmp3['stim1'] = str(val) + str(v)
#                 ls_spikes.append(tmp3)
                
                #df = tmp[tmp.cluster_id==unit]
                #df.trial_st = df.trial_st-t_idx[0]
     
                h, ttr = ena.PSTH(df.times, 0.01, 2.0, 25.0) # all times rescaled to 0-4 this is why trias number 1.0
                zscore = sstat.mstats.zscore(h)
                mean = np.mean(h[:50])
                std = np.std(h[:50])
                if mean==0:
                    std=1
                ztc = (h - mean)/std

                df_psth = pd.DataFrame({     'times':ttr,    'Hz':h, 'cuid': cuid  ,  'et':et, 'fname':fname, 'rec':rec,
                                            'stim1': str(val) + str(v),  'opto':v, 'abs_times': idx*2.0 + ttr + opto_time,
                                        'zscore':zscore, 'ztc':ztc,   'cluster_id':cluster_id, 'exp': exp, 'depth': depth   })
                ls.append(df_psth)
            

    
    df_psth = pd.concat(ls)
  
    df_spikes = 0
    df_tmt = 0
    return df_psth, df_spikes, df_tmt

In [ ]:
d_stim = {}
for i, v in enumerate(short_seq):
    d_stim[i] = v 

In [ ]:
#Receptive field mapping
ls_psth = []
ls_spikes = []
ls_tmt = []
d_stim = {}
d_opto = {}
for i, v in enumerate(short_seq):
    d_stim[i] = v 
for i, v in enumerate(opto_seq):
    d_opto[i] = v 
    
for path in matches[:]:

    print path
    if 'bad' in path:
        continue
    et = path.split('\\')[-2].split()[0]
    rec = path.split('\\')[-2].split()[1]
    stims = sorted([x for x in  get_immediate_subdirectories(os.path.split(path)[0]) if '201' in x])
    
    path_cluster_groups = os.path.join(path, 'cluster_groups.csv')
    cluster_groups = pd.read_csv(path_cluster_groups, sep = '\t')
    # good_units = cluster_groups[cluster_groups.group != 'noise'].cluster_id.values
    noise_units = cluster_groups[cluster_groups['group'] == 'noise'].cluster_id.values
    spike_times = np.load(os.path.join(path, 'spike_times.npy'))
    spike_clusters = np.load(os.path.join(path, 'spike_clusters.npy'))
    templates = np.load(os.path.join(path, 'templates.npy'))
    spike_templates = np.load(os.path.join(path, 'spike_templates.npy'))
    df = pd.DataFrame({'times':spike_times.flatten()/30000.0, 
                       'cluster_id':spike_clusters.flatten(), 
                       'templates':spike_templates.flatten() })

    df = df[~df.cluster_id.isin(noise_units)]
#     df2 = df[df.times <= 160]
#     df3 = df[(df.times > 160)   & (df.times<=940)]
#     df3.times = df3.times - 160

    df4 = df[ (df.times > 940) & (df.times <= 1140)]
    df4.times = df4.times - 940

    df5 = df[df.times > 1340 ]
    df5.times = df5.times - 1340

#     df2['trial_n'] = df2.times//4.0
#     df3['trial_n'] = df3.times//60.0
    df4['trial_n'] = df4.times//2.0
    df5['trial_n'] = df5.times//2.0
#     df_g1 = df2
#     df_rf = df3
    df_kic = df4
    df_kic2 = df5
    
    df_kic_psth, df_kic_spikes, df_kic_tmt  = make_kic_psth(df4, stims[3], et, rec,  d_stim, d_opto, templates, channel_groups)

    ls_psth.append(df_kic_psth)
    
    df_kic2_psth, df_kic2_spikes, df_kic2_tmt  = make_kic_psth(df5, stims[4], et, rec, d_stim, d_opto, templates, channel_groups)
    ls_spikes.append(df_kic2_spikes)
    ls_tmt.append(df_kic2_tmt)
    ls_psth.append(df_kic2_psth)
# df_tmt = pd.concat(ls_tmt)
# df_spikes = pd.concat(ls_spikes)
master_kic = pd.concat(ls_psth)
# master_kic.head()

### Load data

In [ ]:
# path = r"U:\Data_Analysis\pak6\Analysis of units\Kanizsa paper ephys\kic-archt-all-recs.pkl"
# df_psth = pd.read_pickle(path)

# path = r"U:\Data_Analysis\pak6\Analysis of units\Kanizsa paper ephys\kic2-archt-880unit_spikes.pkl"
# df_spikes = pd.read_pickle(path)

# path = r"U:\Data_Analysis\pak6\Analysis of units\Kanizsa paper ephys\rf_maps_kic.pkl"
# rf_df = pd.read_pickle(path)

path = r"U:\Data_Analysis\pak6\Analysis of units\Kanizsa paper ephys\kic_5km.pkl"
kic_5km = pd.read_pickle(path)

# path = r"U:\Data_Analysis\pak6\Analysis of units\Kanizsa paper ephys\kic2-archt-880unit_tmt.pkl"
# df_templates = pd.read_pickle(path)

In [ ]:
kic_5km.cluster_id.unique().size

In [ ]:
def add_kic_index(df, val = 'Hz'):
    tmp_10 = df[df.stim1 == '10']
    tmp_30 = df[df.stim1 == '30']
    tmp_40 = df[df.stim1 == '40']
    base = np.mean(tmp_10[val][:50])
    ind_kic_fr = np.mean(tmp_10[val][55:100])
    kic_fr = np.mean(tmp_10[val][105:150])
    try:    
        kic_ind = (kic_fr - ind_kic_fr)/ (kic_fr + ind_kic_fr)
    except:
        kic_ind = np.nan
    df['kic_ind'] = kic_ind
    
    tmp_20 = df[df.stim1 == '20']
    ind_rot_fr = np.mean(tmp_20[val][55:100])
    rot_fr = np.mean(tmp_20[val][105:150])
    try:
        kic_rot = (kic_fr - rot_fr) / (rot_fr + kic_fr)
    except:
        kic_rot = np.nan
    df['kic_rot'] = kic_rot
    
    tmp_11 = df[df.stim1 == '11']
    archt_fr = np.mean(tmp_11[val][105:150])
    try:
        opto_mod = (kic_fr - archt_fr) / (kic_fr + archt_fr)
    except:
        opto_mod = np.nan
    df['opto_mod'] = opto_mod
    kic_sig = False

    p_ind = sstat.wilcoxon(tmp_10[val][5:50], tmp_10[val][55:100])
#         p_rot = sstat.wilcoxon(tmp_10[val][105:150], tmp_20[val][105:150])
    if p_ind[1] < 0.05:
        kic_sig = True
    else:
        kic_sig = False

    df['ind_sig'] = kic_sig
    return df

In [ ]:
def add_kic_ori_index(df, val = 'Hz'):
    tmp_ori = df[df.stim1.str.contains('10')]
    for stim1 in sorted(tmp_ori.stim1.unique()):
        tmp = df[df.stim1 == stim1]
        base = np.mean(tmp[val][5:35])
        ind_fr = np.mean(tmp[val][55:85])
        ic_fr = np.mean(tmp[val][105:135])

        p_ind = sstat.wilcoxon(tmp[val][5:50], tmp[val][55:100])
        if p_ind[1] < 0.05:
            ind_sig = True
        else:
            ind_sig = False
            
        try:    
            ind_idx = (ic_fr - ind_fr)/ (ic_fr + ind_fr)
        except:
            ind_idx = np.nan
        if ind_sig == False and ind_idx > 0:
            df[stim1 + "_" + 'ic_mod'] = True
        else:
            df[stim1 + "_" + 'ic_mod'] = False
    return df

In [ ]:
kic_sqr_ori_psth = kic_sqr_ori_psth.groupby('cluster_id').apply(add_kic_ori_index)

In [ ]:
test = kic_5km.groupby('cluster_id').apply(add_kic_index)

In [ ]:
bad_inj_et = ['433', '434', '437']
good_recs = test[(test.abs_times == 0) 
     & (test.ind_sig == True)
    & (test.kic_ind > 0)
    & test.kic_rot > 0].groupby(['et', 'rec', 'fname']).cluster_id.count().reset_index()
# good_recs = good_recs[~good_recs.et.isin(bad_inj_et)]
# good_recs = good_recs[good_recs.cluster_id>3]
good_recs = good_recs[good_recs.fname.str.startswith('005')].fname.unique()


In [ ]:
good_recs

In [ ]:
kic_5km.to_pickle('kic_paper_data.pkl')

###  k-means clustering

In [ ]:
from sklearn import metrics
from scipy.spatial.distance import cdist

In [ ]:
key_pca = 'r_groups'
time_idx = np.concatenate((np.arange(100), np.arange(200, 300), np.arange(400, 500), np.arange(600, 700)))
# time_idx = np.arange(50,100)
kic_5km = ena.unit_kmeans(test[test.fname.isin(good_recs)].dropna(), 6, 'abs_times', 'cluster_id', key_pca, time_idx)
# kic_5km = ena.unit_kmeans(kic_psth[kic_psth.exp == '005'], 5, 'abs_times', 'cluster_id', key_pca, time_idx)

In [ ]:
tmp = test[test.fname.isin(good_recs)].dropna()
df_new = tmp.pivot(index= 'abs_times', columns= 'cluster_id', values= 'zscore')
time_idx = np.arange(0, 100)
df_new = df_new.reset_index().drop( 'abs_times',1)
# df_new = df_new.dropna()
df_new = df_new.T
df_new = df_new.dropna()
X = df_new.ix[:,time_idx].values #0.5-2 second interval
y = df_new.index.values.tolist() # corresponding cuid

distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
# plt.savefig('elbow_method_kmeans.pdf', transparent = True)

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

# Instantiate the clustering model and visualizer
model = KMeans()
visualizer = KElbowVisualizer(
    model, k=(4,12), metric='calinski_harabaz', timings=False
)

visualizer.fit(X)    # Fit the data to the visualizer
visualizer.poof()    # Draw/show/poof the data

In [ ]:
tmp = data
tmp = tmp[(tmp.abs_times == 0)]
# tmp = tmp[
# #     (tmp.kic_ind > 0) 
# #             & (tmp.kic_rot > 0)
#              (tmp.kic_sig == False)
# #             & (test.opto_mod < 0)
#            ]
tmp = tmp[['cluster_id', 'kic_rot', 'kic_ind']]
g = sns.jointplot(x = 'kic_ind', y = 'kic_rot', data = tmp, color = 'k', 
                  s = 20, xlim= (-1,1), ylim = (-1, 1))
# g.plot_joint(sns.kdeplot, n_levels = 1, z_order = 0)
# plt.axhline(y=0, linestyle = '--')
# plt.axvline(x=0, linestyle = '--')
# plt.savefig('scatter_im_opto.pdf', transparent = True)

In [ ]:
import matplotlib.gridspec as gridspec

In [ ]:

gr = kic_5km[:
#              (df_psth.stim1 == 1) 
#             & (df_psth.depth < 600) 
#             & (df_psth.et == '494') 
            ]
# gr = gr[gr.times<2.5]
gr = gr.dropna()
bad_units = gr[(gr.ztc>100) | (gr.ztc < -10)].cluster_id.unique()
# gr = gr[gr.times < 2.5]
gr = gr[~gr.cluster_id.isin(bad_units)]
n = int(gr.r_groups.max()+1)
gr = gr[(gr.stim1.str.contains('0'))]
# gr = gr[gr.cluster_id.isin(result[result.n_type == 'fs'].cluster_id.unique())]
f, ax = plt.subplots(n, sharex=True,figsize = (8,12))
# gr = gr[(gr.stim1.str.startswith('3')) | (gr.stim1.str.startswith('4')) ]
# cbar_ax = f.add_axes([.91, .3, .03, .5])
sns.set_style("ticks")
for idx, val in enumerate(sorted(gr.r_groups.unique())):
    tmp = gr[gr.r_groups==val].pivot('cluster_id', 'abs_times', 'zscore').dropna()
    tmp2 = tmp.values[ np.argsort(np.mean(tmp.values[:,50:100], axis = 1) )]
    g = sns.heatmap(tmp2, cmap = 'jet',  
                 ax = ax[idx], xticklabels=200, yticklabels=False, vmax= 7,  vmin = -1, robust = True,
                cbar = False
                   )
    ax[idx].set(xlabel='', ylabel=tmp.index.size )

#     ax[i].set_title('Cluster group ' + str (i+1), loc = 'left') 
f.subplots_adjust(hspace=0.2) 
# ax[0].set_title(str(gr.stim1.unique()[0])  )
# plt.savefig('hm_kic_5km_all_groups.png')

In [ ]:
zsc = 'Hz'
# gr = df_out
# gr = df_out
f, ax = plt.subplots(n, sharex=True,figsize = (6,12))
colors = my_pal[:]
colors = colors[::2]
ts_input = gr[(gr.times > 0.1) & (gr.times<1.9)].sort_values(by=['stim1'])
ts_input = ts_input.sort_values(by = 'stim1')
# colors = ['k', 'g']
for idx, val in enumerate(sorted(gr.r_groups.unique())):
    tmp = ts_input[ts_input.r_groups==val].pivot('cluster_id', 'abs_times', 'zscore').dropna()
    tmp2 = ts_input[ts_input.cluster_id.isin(tmp.index.values)]
    sns.tsplot(tmp2[tmp2.r_groups==val].reset_index(), time = 'times', value = zsc, color = colors, ci = 68,
       unit = 'cluster_id', condition = 'stim1',   estimator=np.nanmean, ax = ax[idx], legend= False,
              )

# plt.axhline(y=0.002,xmin=0,xmax=3,c="black",linewidth=1,zorder=0, ls='dashed')
# plt.xlim(0, 1)
# plt.xlabel('Time (s)')
plt.axvspan(0.4, 1.2,ymin = 0.95, ymax = 1, alpha=0.5, color='green')
plt.axvspan(0.5, 1, alpha = 0.1, color='k')
plt.ylabel(zsc)
sns.despine()
# sns.despine(offset=5, trim=True);
# plt.savefig('line_5km_all_groups.pdf')


In [ ]:
# %matplotlib notebook
f, ax = plt.subplots(figsize = (12, 6), facecolor = 'w')
# data = gr
# data = kic_5km[
#             #(test.kic_ind > 0) 
#             #& (test.kic_rot > 0)
#             #& (test.kic_sig == True)
#             #&  (test.opto_mod < 0)
#      kic_5km.r_groups > -1
#            ]

data = test[test.fname.isin(good_recs)]
data = tmp_psth0[tmp_psth0.deg == tmp_psth0.pref]
data['uet'], data['side'] = data['et'].str.split('-', 1).str
data = data[data.uet == data.uet.unique()[2]]
print data.uet.unique()
# data = data[data.cluster_id.isin(result[result.n_type == 'fs'].cluster_id.unique())]

# data = test[test.fname.isin(good_recs)].dropna()
# bad_units = data[(data.ztc>100) | (data.ztc < -10)].cluster_id.unique()
# data = data[~data.cluster_id.isin(bad_units)]
# data = data[(data.depth < 500)]
data = data[(data.stim1.str.contains('1')) 
            | (data.stim1.str.contains('2')) 
           ]
data['stim_times'] = data['times'] + data['stim'].astype(int)*2

# data = data[(data['kic_ind'] > 0) 
           
# #             & (data.kic_rot > 0.1)
#             & (data['ind_sig'] == False)
#            ]
hm = data.pivot('cluster_id', 'stim_times', 'zscore')
hm = hm.dropna()
hm2 = hm.values[ np.argsort(np.mean(hm.values[:,100:150], axis = 1) - np.mean(hm.values[:,50:100], axis = 1) )]
# hm2 = hm2[-100:]
# plt.figure
# plt.figure(figsize=(20,20))
sns.heatmap(hm2, cmap = 'jet',  annot=False, xticklabels=  200, vmax = 7, cbar = False,
            vmin = -1, robust = True, yticklabels=False, ax = ax )
ax.set(xlabel='', ylabel= hm2.shape[0])
# plt.savefig('hm_ic_ind-sig_12opto.png')

In [ ]:
plt.figure(figsize=(12, 6))
# gr = df_psth[
#             (df_psth.stim1 == 'sqr-line') |
# #             & (df_psth.cluster_id == '100et495pre') 
#              (df_psth.stim1 == 'sqr') 
# #             & (df_psth.depth > 500)
#             ]
zsc = 'Hz'
colors = my_pal[::2]
# colors = colors[4:]

data_tmp = data[data.cluster_id.isin(hm.index.values)]
# data_tmp = data_tmp[(data_tmp.stim1.str.startswith('1')) | (data_tmp.stim1.str.startswith('2')) ]
# data_tmp = data_tmp[(data_tmp.stim1.str.contains('0'))]


ts_input = data_tmp[(data_tmp.times > 0.1) & (data_tmp.times<1.9)].sort_values(by=['stim1'])
ts = sns.tsplot(ts_input.reset_index(), time = 'times', value = zsc, color  = colors, legend = False,
           unit = 'cluster_id',   estimator=np.nanmean, condition = 'stim', ci = 68,
          )

# plt.axvspan(0.9, 1.7,ymin = 0.97, ymax = 1, alpha=0.5, color='green')
# plt.axvspan(0.9, 1.7, alpha = 0.05, color='green')
plt.axvspan(1, 1.5, alpha = 0.1, color='k')
plt.axvspan(0.5, 1, alpha = 0.05, color='k')

# ts.set(xlim=(0.4, 0.5))
sns.despine()
# plt.savefig('line_ind_sig_12opto.pdf')


In [ ]:
fig_inp = data_tmp[
                 (data_tmp.times > 1.05) 
                & (data_tmp.times < 1.5)].dropna()

fig_inp = fig_inp.groupby([ 'stim1' ,'cluster_id']).mean().reset_index().dropna() 

g = sns.factorplot(  x = 'stim1', y= "Hz", data= fig_inp, kind = 'bar', ci = 68,
                   palette = colors, 
#                    capsize = 0.02, saturation = 1,
                   size = 6, aspect = 1 )
# g = sns.swarmplot(x = 'stim1', y = 'Hz', data = fig_inp, color = '0.25', alpha = 0.5)
# plt.ylim(0, 20)
# plt.savefig('bar_kic_ind-sig_12opto.pdf', transparent = True)

In [ ]:
x1 = _inp[ (_inp.layer == 'l2/3')].opto_mod.dropna().values
x2 = _inp[ (_inp.layer == 'l4')].opto_mod.dropna().values
x3 = _inp[ (_inp.layer == 'l5/6')].opto_mod.dropna().values
print np.mean(x1), sstat.sem(x1)
print np.mean(x2), sstat.sem(x2)
print np.mean(x3), sstat.sem(x3)
print len(x1), len(x2), len(x3)

print sstat.mannwhitneyu(x1, x2)
print sstat.mannwhitneyu(x1, x3)
print sstat.mannwhitneyu(x2, x3)



In [ ]:
x1 = fig_inp[(fig_inp.r_groups == 0) & (fig_inp.opto == 0)].Hz.values
x2 = fig_inp[(fig_inp.r_groups == 1) & (fig_inp.opto == 0)].Hz.values
x3 = fig_inp[(fig_inp.r_groups == 2) & (fig_inp.opto == 0)].Hz.values
x4 = fig_inp[(fig_inp.r_groups == 3) & (fig_inp.opto == 0)].Hz.values

y1 = fig_inp[(fig_inp.r_groups == 0) & (fig_inp.opto == 1)].Hz.values
y2 = fig_inp[(fig_inp.r_groups == 1) & (fig_inp.opto == 1)].Hz.values
y3 = fig_inp[(fig_inp.r_groups == 2) & (fig_inp.opto == 1)].Hz.values
y4 = fig_inp[(fig_inp.r_groups == 3) & (fig_inp.opto == 1)].Hz.values

print np.mean(x1), sstat.sem(x1)
print np.mean(y1), sstat.sem(y1)

print np.mean(x2), sstat.sem(x2)
print np.mean(y2), sstat.sem(y2)
print np.mean(x3), sstat.sem(x3)
print np.mean(y3), sstat.sem(y3)
print np.mean(x4), sstat.sem(x4)
print np.mean(y4), sstat.sem(y4)

print len(x1), len(x2), len(x3), len(x4)
print len(y1), len(y2), len(y3), len(y4)
# print sstat.kruskal(x1, x2, x3, x4)

print sstat.wilcoxon(x1, y1, zero_method='pratt', correction=True)
print sstat.wilcoxon(x2, y2, zero_method='pratt', correction=True)
print sstat.wilcoxon(x3, y3, zero_method='pratt', correction=True)
print sstat.wilcoxon(x4, y4, zero_method='pratt', correction=True)


In [ ]:
x1 = fig_inp[fig_inp.stim1 == '10'].dropna().Hz.values
x2 = fig_inp[fig_inp.stim1 == '11'].dropna().Hz.values
x3 = fig_inp[fig_inp.stim1 == '20'].dropna().Hz.values
x4 = fig_inp[fig_inp.stim1 == '21'].dropna().Hz.values

print np.mean(x1), sstat.sem(x1)
print np.mean(x2), sstat.sem(x2)
print np.mean(x3), sstat.sem(x3)
print np.mean(x4), sstat.sem(x4)
print len(x1), len(x2), len(x3), len(x4)
n = min(len(x1), len(x2), len(x3), len(x4))

print sstat.kruskal(x1, x2, x3, x4)

print len(x1)

print sstat.mannwhitneyu(x1, x2)
print sstat.mannwhitneyu(x1, x3)
print sstat.mannwhitneyu(x1, x4)
print sstat.mannwhitneyu(x2, x3)
print sstat.mannwhitneyu(x2, x4)
print sstat.mannwhitneyu(x3, x4)

print sstat.wilcoxon(x1[:n], x2[:n], zero_method='pratt', correction=True)
print sstat.wilcoxon(x1[:n], x3[:n], zero_method='pratt', correction=True)
print sstat.wilcoxon(x1[:n], x4[:n], zero_method='pratt', correction=True)
print sstat.wilcoxon(x2[:n], x3[:n], zero_method='pratt', correction=True)
print sstat.wilcoxon(x2[:n], x4[:n], zero_method='pratt', correction=True)
print sstat.wilcoxon(x3, x4, zero_method='pratt', correction=True)

In [ ]:
min(len(x1), len(x2), len(x3), len(x4))

In [ ]:
_inp = fig_inp.groupby([ 'cluster_id', 'stim1']).mean().reset_index()
_data = _inp.pivot(index='cluster_id', columns='stim1', values='Hz').reset_index()
#_data = _data[['cluster_id', '10', '11']]
lims = [0, 20]
_data = _data[(_data>lims[0]) & (_data<lims[1])]
g = sns.jointplot(x = '41', y = '40', data = _data, color = 'k', 
                  s = 30, stat_func = None,
                  xlim= lims, ylim = lims
                 )
g.plot_joint(sns.kdeplot, n_levels = 2, z_order = 0)

x0, x1 = g.ax_joint.get_xlim()
y0, y1 = g.ax_joint.get_ylim()
lims = [max(x0, y0), min(x1, y1)]
g.ax_joint.plot(lims, lims, '--k', linewidth = 3 )    
# plt.savefig('scatter_ind_sig_opto_41.pdf', transparent = True)

In [ ]:
conditions = [
    (data_tmp['depth'] <= 600) & (data_tmp['depth'] >= 400),
    (data_tmp['depth'] < 400) ,
    (data_tmp['depth'] > 600)]
choices = ['l4', 'l5/6', 'l2/3']
data_tmp['layer'] = np.select(conditions, choices)

In [ ]:
data_tmp.cluster_id.unique().size

In [ ]:
# f, ax = plt.subplots()
bins = np.arange(0, 1001, 50)
labels = bins[1:]
_inp = data_tmp[(data_tmp.opto_mod > -10) & (data_tmp.abs_times == 4.5)]
_inp['bin_depth'] = pd.cut(_inp['depth'], bins=bins, labels=labels)
_inp = _inp.sort_values(by = 'layer')
sns.despine()
sns.catplot(data = _inp, x = 'layer', y = 'kic_rot',  color = 'gray',
            kind= 'box', height = 5, aspect = 1 )

plt.savefig('box_kic_rot_layer.pdf', transparent = True)

In [ ]:
dir_seq = [10, 7, 3, 2, 4, 8, 9, 5, 7, 3, 4, 8, 3, 2, 1, 8, 0, 4, 9, 11, 
10, 9, 1, 11, 4, 0, 7, 1, 2, 8, 2, 9, 11, 9, 6, 5, 10, 4, 9, 0, 7, 11, 9, 
5, 9, 10, 11, 6, 8, 9, 5, 4, 2, 8, 11, 2, 10, 3, 5, 1, 7, 0, 4, 9, 1, 5, 
11, 3, 5, 10, 1, 2, 9, 6, 2, 2, 11, 5, 10, 7, 3, 7, 4, 6, 8, 4, 1, 8, 0, 
11, 0, 6, 2, 11, 1, 10, 3, 8, 3, 1, 2, 10, 5, 3, 11, 1, 7, 3, 4, 7, 8, 4, 6, 
7, 11, 7, 0, 8, 6, 10, 4, 5, 7, 2, 10, 3, 5, 9, 8, 6, 3, 2, 0, 11, 0, 6, 10, 
0, 7, 4, 5, 0, 10, 6, 8, 10, 3, 11, 9, 0, 5, 1, 3, 7, 0, 6, 9, 1, 6, 10, 5, 
6, 11, 7, 0, 5, 1, 4, 1, 6, 8, 2, 9, 2, 8, 3, 0, 4, 6, 1]

In [ ]:
_inp

In [ ]:
f, ax = plt.subplots(1, figsize = (8,12))
sns.despine()
ax.plot(b[0]/a[0], bins[1:])
# plt.savefig('line_norm_depth_dist.pdf', transparent = True)

In [ ]:
bins = np.arange(0, 1001, 50)
tmp = kic_5km.groupby('cluster_id').depth.mean().values
a = np.histogram(tmp, bins= bins)
dep_gr0 = data_tmp.groupby('cluster_id').depth.mean().values
b = np.histogram(dep_gr0, bins= bins)
a

In [ ]:

dep_gr0 = data_tmp.groupby('cluster_id').depth.mean().values
sns.distplot(dep_gr0, color = 'k', bins = np.arange(0, 1025, 50), kde = False)
# sns.distplot(dep2, color = 'cyan')
sns.despine()
plt.gca().invert_xaxis()
#     plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.xlim(1200, -200)

# plt.savefig('dist_depth.pdf', transparent = True)
# plt.xlim(-150, 1200)

In [ ]:
dep_gr0 = df_rf_size[(df_rf_size.on > 0) & (df_rf_size.r_groups == 0)].on.values
dep_gr1 = df_rf_size[(df_rf_size.on > 0) & (df_rf_size.r_groups == 1)].on.values
dep_gr2 = df_rf_size[(df_rf_size.on > 0) & (df_rf_size.r_groups == 2)].on.values
dep_gr3 = df_rf_size[(df_rf_size.on > 0) & (df_rf_size.r_groups == 3)].on.values
dep_gr4 = df_rf_size[(df_rf_size.on > 0) & (df_rf_size.r_groups == 4)].on.values

print len(dep_gr0), len(dep_gr1), len(dep_gr2), len(dep_gr3), len(dep_gr4)
print np.mean(dep_gr0), np.mean(dep_gr1), np.mean(dep_gr2), np.mean(dep_gr3), np.mean(dep_gr3)
print sstat.kruskal(dep_gr0, dep_gr1, dep_gr2, dep_gr3, dep_gr3)

print sstat.ks_2samp(dep_gr0, dep_gr1)
print sstat.ks_2samp(dep_gr0, dep_gr2)
print sstat.ks_2samp(dep_gr0, dep_gr3)
print sstat.ks_2samp(dep_gr0, dep_gr4)

print sstat.ks_2samp(dep_gr1, dep_gr2)
print sstat.ks_2samp(dep_gr1, dep_gr3)                                                                                                                      
print sstat.ks_2samp(dep_gr1, dep_gr4)

print sstat.ks_2samp(dep_gr2, dep_gr3)
print sstat.ks_2samp(dep_gr2, dep_gr4)

print sstat.ks_2samp(dep_gr3, dep_gr4)

In [ ]:
data

### Latency analysis

In [ ]:
ls = []
from detect_peaks import detect_peaks
_data = data
idx = 0
for unit in _data.cluster_id.unique():
    tmp = _data[_data.cluster_id == unit]
    
    for stim in tmp.stim1.unique():
        tmp2 = tmp[tmp.stim1 == stim].Hz.values
        
        thresh = np.mean(tmp2[:50]) + 2 * np.std(tmp2[:50])
        idx=+1
        try:
            peak_0 = detect_peaks(tmp2[50:80], mph = thresh, mpd = 30)[0]
        except:
            peak_0 = np.nan
        try:
            peak_1 = detect_peaks(tmp2[100:130], mph = thresh, mpd = 30)[0]
        except:
            peak_1 = np.nan    
   
        out = pd.DataFrame({'cluster_id': unit, 'stim1':stim, 'lnc_4c':peak_0,
                           'lnc_stim': peak_1 }, index = [idx])
        ls.append(out)

#         plt.plot(tmp2)

#         plt.axvline(x=peak_1+100)
#         plt.show()
lnc_df = pd.concat(ls)

In [ ]:
# lnc_df['lnc_stim'] = lnc_df['lnc_stim'] + 10
g = sns.factorplot(x = "stim1", y ="lnc_stim", data = lnc_df,  kind = 'point', ci = 68,
                  palette = colors, 
#                    capsize = 0.02, saturation = 1,
                   size = 4, aspect=1 
                  )
print lnc_df.groupby('stim1').lnc_stim.mean()
plt.ylim(12, 22)
# g.invert_yaxis()  
# plt.savefig('point_peak_time.pdf'.format(stim))

In [ ]:
# sns.set_palette(colors)
for idx, stim in enumerate(lnc_df.stim1.unique()[:]):
    _inp = lnc_df[(lnc_df.stim1 == stim)].lnc_stim.dropna()
    sns.kdeplot(_inp.values, cumulative=True, color = colors[idx])
#     plt.title( stim + 'n' + str(_inp.values.size))
    plt.xlim(0, 35)
    sns.despine()
    plt.tight_layout()
# plt.savefig('cdf_peak_time.pdf'.format(stim))

In [ ]:
pt1 = lnc_df[(lnc_df.stim1 == '10')].lnc_stim.dropna()
pt2 = lnc_df[(lnc_df.stim1 == '20')].lnc_stim.dropna()
pt3 = lnc_df[(lnc_df.stim1 == '30')].lnc_stim.dropna()
pt4 = lnc_df[(lnc_df.stim1 == '40')].lnc_stim.dropna()

print np.mean(pt1)/100, sstat.sem(pt1)/100
print np.mean(pt2)/100, sstat.sem(pt2)/100
print np.mean(pt3)/100, sstat.sem(pt3)/100
print np.mean(pt4)/100, sstat.sem(pt4)/100

print len(pt1), len(pt2), len(pt3), len(pt4)

print sstat.kruskal(pt1, pt2, pt3, pt4)

print sstat.mannwhitneyu(pt1, pt2)
print sstat.mannwhitneyu(pt1, pt3)
print sstat.mannwhitneyu(pt1, pt4)
print sstat.mannwhitneyu(pt2, pt3)
print sstat.mannwhitneyu(pt2, pt4)
print sstat.mannwhitneyu(pt3, pt4)

In [ ]:
df_templates.head()

In [ ]:
import resampy
from copy import deepcopy
import scipy.optimize as opt
from scipy.ndimage.filters import gaussian_filter
def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

In [ ]:
result = df_templates
spk_width = {}
d_fwhm = {}
tr2peak = {}
neuron_type = {}
ls = []
ls2 = []
ls3 = []
# f, ax = plt.subplots(1,2)
for ii in result.cluster_id.unique()[::]:
#trough-to-peak    
    #tp =  result[(result['cuid'] == ii)  & (result.index > 18)  ].tmt.idxmax() - result[(result['cuid'] == ii)  & (result.index == 18)  ].index
    tmt_data = np.array(result[(result['cluster_id'] == ii)   ].tmt)

    y = resampy.resample( tmt_data[::-1] , 1 ,10,  filter='sinc_window',
                                    num_zeros=10, precision=5,
                                    window=ssig.hann)
    trough_idx = y.argmin()
    peak_idx = y[:y.argmin()].argmax()
#     plt.plot(y)
#     plt.axvline(x= y.argmin(), color = 'k', linestyle = '--')
#     plt.axvline(x= y[:y.argmin()].argmax(), color = 'r',linestyle = '--')
#     plt.show()

    tp = abs((y.argmin() - y[:y.argmin()].argmax())/300.0)
    
    x = np.arange(y.size)
    y_gaus = y*(-1)
    popt,pcov = opt.curve_fit(gaus,x,y_gaus,p0=[0.2, y.argmin(), 10])
    fwhm = popt[-1]/300*2.355
    
#     plt.plot(x,y*(-1),'b+:')
#     plt.plot(x,gaus(x,*popt),'r--')
#     plt.show()

    f,pxx = ssig.welch(tmt_data, fs=3e4,  nfft=5096,  nperseg=48,
                          return_onesided=True, scaling='spectrum')

    df = np.vstack((f, pxx))
    df = pd.DataFrame(df)
    idx = df.T[1].idxmax()
    w = df.T[0][idx]
    w = 1/w*1000.0

    ls2.append(w)
    ls.append(tp)
    
    spk_width[ii] = w
    tr2peak[ii] = tp
    d_fwhm[ii] = fwhm
    if tp<0.4:
        neuron_type[ii] = 'fs'
    else:
        neuron_type[ii] = 'rs'

#p/t ratio
    edge = 100
    if peak_idx < 100:
        edge = peak_idx
    y_slope = y[peak_idx-edge:peak_idx]
    x_slope = np.arange(y_slope.size)
    slope, intercept, r_value, p_value, std_err = sstat.linregress(x_slope, y_slope)
    ls3.append(1e3*(slope))
    
result['n_type'] = result.cluster_id.map(neuron_type)
result['sp_w'] = result.cluster_id.map(spk_width)
result['tp'] = result.cluster_id.map(tr2peak)


In [ ]:
g = (sns.jointplot(np.array(ls2), np.array(ls), stat_func= None,
             color="k", s = 20)
       .plot_joint(sns.kdeplot, zorder=0, n_levels=6))
g.set_axis_labels('Spike width [ms]', 't-p width [ms]')
# plt.axhline(y= 0.45, linestyle = '--')
plt.axvline(x= 1.1,  linestyle = '--')
plt.axhline(y= 0.4, linestyle = '--')

In [ ]:
total = result.cluster_id.unique().size
print total
print result[result.index == 0].groupby('n_type').cluster_id.count()/total

In [ ]:
result = result.reset_index()
result[result.n_type == 'rs'].groupby('index').tmt.mean().plot()
result[result.n_type == 'fs'].groupby('index').tmt.mean().plot()

## RF map analysis

In [ ]:
# path = r"U:\Data_Analysis\pak6\Analysis of units\rf_maps_kic.pkl"
# rf_df = pd.read_pickle(path)
d_rf = rf_df.to_dict()

In [ ]:
_inp = kic_5km
for unit in _inp[(_inp.et == '434') & (_inp.rec == 'left')].cluster_id.unique()[:6]:
    print unit
    print _inp[_inp.cluster_id == unit].depth[0].mean()-1000
    try:
        plt.imshow(d_rf['on'][unit], cmap = plt.cm.gray)
        plt.colorbar()
        plt.show()
        plt.imshow(d_rf['off'][unit], cmap = plt.cm.gray)
        plt.colorbar()
        plt.show()
    except:
        continue
# plt.savefig('rf_map_145005et435right.pdf'.format(stim))

In [ ]:
# rf quanitification using opt.curve_fit to 2d-gaus
# d_rf_size = {}
# d_rf_size['on'] = {}
# d_rf_size['off'] = {}
for unit in kic_5km.cluster_id.unique()[14:15]:    
    data_fitted_on = data_fitted_off = data_on = data_off = threshold = popt_on = popt_off = 0
    try: 
        _inp_on = deepcopy(d_rf['on'][unit])[:,30:120]
        _inp_off = deepcopy(d_rf['off'][unit])[:,30:120]
        if np.std(_inp_on) > 2 or np.std(_inp_off)>2:
            continue
            
        f, ax = plt.subplots(2, 3, sharex = True, sharey = True, figsize = (9,6))
        
        ax[0][0].imshow(_inp_on, cmap = 'gray' )
        ax[1][0].imshow(_inp_off, cmap = 'gray' )

#         tmp = np.array(sstat.zscore(_inp,ddof=1))
#         _inp_on = gaussian_filter(_inp_on, sigma=1)
#         _inp_off = gaussian_filter(_inp_off, sigma=1)
        
        tmp_on, threshold = threshold_rf(_inp_on, 0.975)
        tmp_off, threshold = threshold_rf(_inp_off, 0.025)
        

#         ax[1].imshow(tmp, cmap = 'gray')
        if threshold == 0:
            continue
        
        ax[0][1].imshow(tmp_on, cmap='Reds_r' )
        ax[1][1].imshow(tmp_off, cmap='Blues')

        data_on = deepcopy(tmp_on)/255
        data_off = deepcopy(tmp_off)/255

#         data_on = ssig.medfilt2d(data_on, kernel_size=(3,3))
        
        x = np.linspace(0, 89, 90)
        y = np.linspace(0, 89, 90)
        x, y = np.meshgrid(x, y)
       
        ind_on = np.unravel_index(np.argmin(data_on, axis=None), data_on.shape)
        initial_guess_on = (data_on.min(), ind_on[1], ind_on[0], 20, 20, 0, 1)
        popt_on, pcov_on = opt.curve_fit(twoD_Gaussian, (x,y), data_on.ravel(),absolute_sigma=True,
                  p0 = initial_guess_on, bounds = (-np.inf, [1, 90, 90, 40, 40, np.inf, np.inf]))
        perr_on = np.sqrt(np.diag(pcov_on))  
        
        ind_off = np.unravel_index(np.argmax(data_off, axis=None), data_off.shape)
        initial_guess_off = (data_off.max(), ind_off[1], ind_off[0], 20, 20, 0, 1)
        popt_off, pcov_off = opt.curve_fit(twoD_Gaussian, (x,y), data_off.ravel(), absolute_sigma=True,
                       p0 = initial_guess_off, bounds = (-np.inf, [1, 90, 90, 40, 40, np.inf, np.inf]))
        perr_off = np.sqrt(np.diag(pcov_off))

        data_fitted_on = twoD_Gaussian((x, y), *popt_on)
        data_fitted_off = twoD_Gaussian((x, y), *popt_off)

        ax[0][2].imshow(data_fitted_on.reshape(90, 90), cmap='Reds_r')
        ax[1][2].imshow(data_fitted_off.reshape(90, 90), cmap='Blues')
        
        if abs(popt_on[3]) < 1.9 or abs(popt_on[3]) > 20 or abs(popt_on[4]) < 1.9 or abs(popt_on[4]) > 20:
            hwhm_on = np.nan
        else:
            hwhm_on = 2.355*0.5*(abs(popt_on[3]) + abs(popt_on[4]))*0.5*0.96
            ax[0][2].text(0.95, 0.05, """
            err: %.1f
            x : %.1f
            y : %.1f
            width_x : %.1f
            width_y : %.1f""" %(perr_on[3]+perr_on[4], 
                                popt_on[1], popt_on[2], popt_on[3], popt_on[4]),
                    fontsize=14, horizontalalignment='right',
                    verticalalignment='bottom', transform=ax[0][2].transAxes)

        if abs(popt_off[3]) < 1.9 or abs(popt_off[3]) > 20 or abs(popt_off[4]) < 1.9 or abs(popt_off[4]) > 20:
            hwhm_off = np.nan
        else:
            # compute flhm/2 and scale by degree in pixel for donwsa,ple images (12 times)
            hwhm_off = 2.355*0.5*(abs(popt_off[3]) + abs(popt_off[4]))*0.5*0.96
            ax[1][2].text(0.95, 0.05, """
            err: %.1f
            x : %.1f
            y : %.1f
            width_x : %.1f
            width_y : %.1f""" %(perr_off[3]+perr_off[4], popt_off[1], popt_off[2], popt_off[3], popt_off[4]),
                    fontsize=14, horizontalalignment='right',
                    verticalalignment='bottom', transform=ax[1][2].transAxes)
#         d_rf_size['on'][unit] = hwhm_on
#         d_rf_size['off'][unit] = hwhm_off
    except:
        continue

# plt.savefig('rf_size_analysis2.pdf')

In [ ]:
df_rf_size = pd.DataFrame.from_dict(d_rf_size).reset_index()

In [ ]:
import cv2


In [ ]:
windowOpen = np.ones((5,5),np.uint8)
tmp = np.interp(_inp_off, (_inp_off.min(), _inp_off.max()), (0, 255))
equ = cv2.equalizeHist(tmp.astype(np.uint8))
blur = cv2.GaussianBlur(equ, (5,5), 3)
retval, thresh = cv2.threshold(blur.astype(np.uint8), 30, 255, cv2.THRESH_BINARY_INV)
pupilFrame = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, windowOpen)
# threshold = cv2.inRange(pupilFrame,10,255)		#get the blobs
_, contours, hierarchy = cv2.findContours(pupilFrame,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(contours, key = cv2.contourArea, reverse = True)
cnt = cnts[0]
# cv2.drawContours(pupilFrame, [cnt], 0, 120, 1)
try:
    ellipse = cv2.fitEllipse(cnt)
    cv2.ellipse(pupilFrame, ellipse, 120,1)
except:
    print 'no fit'

In [ ]:
plt.imshow(pupilFrame, cmap='jet')

In [ ]:
thresh.mean()

In [ ]:
# path = r"U:\Data_Analysis\pak6\Analysis of units\Kanizsa paper ephys\kic_rf_size.pkl"
# df_rf_size = pd.read_pickle(path)
# df_rf_size.head()

In [ ]:
# f, ax = plt.subplots(figsize = (6,6))
_inp = df_rf_size
g = sns.jointplot(x="on", y="depth", data=_inp[_inp.on > 0], kind = 'kde', 
                  color = 'r', stat_func=None, xlim = (0, 15))
g.plot_marginals(sns.rugplot,height = 0.25, color = 'k' )

# plt.savefig('kde_rf_size_on_depth.pdf')

In [ ]:
df_rf_size[df_rf_size.off > 0].shape[0]

In [ ]:
dep_d = dict(zip(kic_5km.cluster_id, kic_5km.depth))
gr_d = dict(zip(kic_5km.cluster_id, kic_5km.r_groups))
df_rf_size['depth'] = df_rf_size['index'].map(dep_d)
df_rf_size['r_groups'] = df_rf_size['index'].map(gr_d)

In [ ]:
sns.distplot(df_rf_size[(df_rf_size.on > 0) & (df_rf_size.r_groups == 4)].on, color = 'r')
sns.distplot(df_rf_size[(df_rf_size.off > 0) & (df_rf_size.r_groups == 4)].off, color = 'b')
plt.xlim(0, 20)
plt.ylim(0, 0.5)
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
sns.despine()
plt.savefig('dist_rf_size_group4.pdf')

In [ ]:
df_rf_size.to_pickle('kic_rf_size.pkl')

In [ ]:
def threshold_rf(_inp, a):
    hist, bin_edges = np.histogram(_inp.flatten(), density=True)
    for idx, val in enumerate(np.cumsum(hist/hist.sum())):
    #             if val >= 0.03:
        if val >= a:
            threshold = bin_edges[idx]-0.0001
            break

    tmp = deepcopy(_inp)
    tmp = gaussian_filter(tmp, sigma=2)
    idx = tmp > threshold
    tmp[idx] = False
    return tmp, threshold

In [ ]:
_inp = deepcopy(d_rf['off'][unit])
_inp = _inp[:,30:120]
plt.imshow(_inp, cmap = 'gray')
plt.colorbar()
plt.show()
tmp = deepcopy(_inp)
idx = tmp > threshold
tmp[idx] = False
tmp = tmp
# tmp = sstat.zscore(np.ravel(tmp))
plt.imshow(tmp, cmap = 'gray')
plt.colorbar()
plt.show()
#         tmp = ssig.medfilt2d(_inp, kernel_size=(7,3))


In [ ]:
ind_on

In [ ]:
hist, bin_edges = np.histogram(_inp.flatten(), density=True)
for idx, val in enumerate(np.cumsum(hist/hist.sum())):
    if val >= 0.015:
        threshold = bin_edges[idx]-0.0001
        break
print threshold

In [ ]:
def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape)
    x = (X*data).sum()/total
    y = (Y*data).sum()/total
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum())
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum())
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = 100, 5, 5, 10,10
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = opt.curve_fit(gaussian, errorfunction, params)
    return p

In [ ]:
# Create the gaussian data

data = tmp
plt.matshow(data, cmap= plt.cm.gist_earth_r)

params = fitgaussian(data)
fit = gaussian(*params)

plt.contour(fit(*np.indices(data.shape)),1, cmap = plt.cm.copper)
ax = plt.gca()
(height, x, y, width_x, width_y) = params

plt.text(0.95, 0.05, """
x : %.1f
y : %.1f
width_x : %.1f
width_y : %.1f""" %(x, y, width_x, width_y),
        fontsize=16, horizontalalignment='right',
        verticalalignment='bottom', transform=ax.transAxes)

In [ ]:

print ind

In [ ]:
# Create x and y indices
x = np.linspace(0, 89, 90)
y = np.linspace(0, 89, 90)
x, y = np.meshgrid(x, y)

#create data
data = twoD_Gaussian((x,y), data.max(), ind[1], ind[0], 10, 10, 0, 1)

# plot twoD_Gaussian data generated above
plt.figure()
plt.imshow(data.reshape(90, 90))
plt.colorbar()

In [ ]:
d_rf

In [ ]:
# xo,y0 center, 
def twoD_Gaussian((x, y), amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

In [ ]:
popt

In [ ]:
# add some noise to the data and try to fit the data generated beforehand
initial_guess = (data.max()/2, ind[1], ind[0], 10, 10, 0, 1)

popt, pcov = opt.curve_fit(twoD_Gaussian, (x,y), data.ravel(), p0=initial_guess)

In [ ]:
data_fitted = twoD_Gaussian((x, y), *popt)

fig, ax = plt.subplots(1, 1)
ax.hold(True)
ax.imshow(data.reshape(90, 90), cmap=plt.cm.gray, origin = 'bot',
    extent=(x.min(), x.max(), y.min(), y.max()))
ax.contour(x, y, data_fitted.reshape(90,90), 1, colors='r')
ax.invert_yaxis()

## Analysis of LFP

In [ ]:
exclude_keyphrases=['Bad Reports','bad', 'MSF', 'phy']
directory = r"U:\Data\pak6\figures\LFP\probe_64DB\wt\illusory_contours\LM-ArchT"
matches = [os.path.join(dirpath, f)
            for dirpath, dirnames, files in os.walk(directory)
            for f in files 
            if (f.endswith('.h5') and not any(filter(lambda bad: bad in f, exclude_keyphrases)))]
matches = [f for f in matches if 'bad' not in f and '201' in f]
matches[:5]

In [ ]:
for fname in matches[:]:
    et = fname.split('\\')[-1].split()[0][1:]
    rec = fname.split('\\')[-1].split()[1].split('_')[0]
    paradigm = fname.split('\\')[-1].split()[-1].split('_')[-6]
    exp =  fname.split('\\')[-1].split()[-1].split('_')[1]
    

In [ ]:
ls2 = []

for fname in matches[:]:
    et = fname.split('\\')[-1].split()[0][1:]
    rec = fname.split('\\')[-1].split()[1].split('_')[0]
    paradigm = fname.split('\\')[-1].split()[-1].split('_')[-6]
    exp =  fname.split('\\')[-1].split()[-1].split('_')[1]
    print fname
    ls = []
       
    try:
        tmp =  pd.read_hdf(fname, key = 'raw') 
#         trial_length = pd.read_hdf(fname, key = 'trial_duration') 
    except:
        continue
    trial_length = 2000
    trials =  200
    times = np.linspace(0, trial_length/1e3, trial_length)
    df3_array = np.reshape(tmp.values,(np.shape(tmp)[0],trials, -1))
    df3_array = df3_array[:,np.where((opto_seq==0)  & (trial_seq == 1 ))[0],:]
    print df3_array.shape
    
    df3_avg=np.mean(df3_array,1)
    df3_avg = pd.DataFrame(df3_avg).T        
    df3_avg['times'] = times
    negativity_ch_idx = df3_avg[(df3_avg['times']>= 0.55) & (df3_avg['times']< 0.7)].ix[:,8:16].min().idxmin()
#     negativity_ch_idx = negativity_ch_idx - 3
    print negativity_ch_idx

    # plotting the spectra and FFR for the maximum negativity channel, the first trial
    
    ddf = tmp.values[negativity_ch_idx,:]
    
    ddf2 = np.reshape(ddf,(trials, -1))
    print ddf2.shape
    ddf2 = pd.DataFrame(ddf2)
    for i in np.unique(opto_seq):
        
        for j in np.unique(trial_seq):

            sel_idx = np.where((opto_seq == i) & (trial_seq == j))[0]
            lfp = ddf2.iloc[sel_idx].mean()

            tmp_df = pd.DataFrame({'et': et, 'lfp':lfp, 'paradigm': paradigm,  'stim1':j, 'rec':rec, 'exp':exp,
                                   'fname':fname, 'times':times, 'opto':i })
            ls.append(tmp_df)
    tmp_df2 = pd.concat(ls)
    ls2.append(tmp_df2)
    
kic_lfp = pd.concat(ls2)
    

In [ ]:
kic_lfp.to_pickle('kic-lfp_l4.pkl')

In [ ]:
path = r"U:\Data_Analysis\pak6\Analysis of units\kic2_archt\kic-lfp_sup.pkl"
lfp_sup = pd.read_pickle(path)

path = r"U:\Data_Analysis\pak6\Analysis of units\kic2_archt\kic-lfp_l4.pkl"
lfp_l4 = pd.read_pickle(path)

path = r"U:\Data_Analysis\pak6\Analysis of units\kic2_archt\kic-lfp_deep.pkl"
lfp_deep = pd.read_pickle(path)


### Plotting traces

In [ ]:
opto_recs = lfp_deep.groupby('fname').min().reset_index()
x = opto_recs['et'] + opto_recs['exp'] + opto_recs['rec']
x.values

In [ ]:

plt.figure(figsize=(12, 8))
_inp = lfp_sup[lfp_sup.opto == 0]
# opto_recs = _inp.groupby('fname').lfp.min().reset_index()
# opto_recs = opto_recs[opto_recs.lfp > -700].fname.unique()
# _inp = _inp[_inp.fname.isin(opto_recs)]
colors = my_pal
colors = colors[::2]
_inp = _inp.sort_values(by=['stim1'])
# _inp = _inp[_inp.et.str.startswith('7')]
ts = sns.tsplot(_inp.reset_index(), time = 'times', value = 'lfp', color  = colors,
           unit = 'fname',   estimator=np.nanmean, condition = 'stim1'
          )


plt.axvspan(1, 1.5, alpha = 0.2, color='k')
plt.axvspan(0.5, 1, alpha = 0.1, color='k')

#ts.set(xlim=(1,2))
#ts.set(ylim=(-200,200))
sns.despine()
# plt.savefig('sup_vep_opto_kic.pdf')
# plt.savefig('sf_tc_post.png')

In [ ]:
_inp.fname.unique().size

In [ ]:
#_inp = master_df[master_df.training=='pre'].reset_index()
_inp = lfp_sup[lfp_sup.opto == 1
              ]

vep1 = _inp[(_inp.times > 0.55) & (_inp.times < 1)]
vep1 = vep1.ix[vep1.groupby([ 'fname' ,'stim1']).lfp.idxmin().values]
vep1 = vep1.groupby([ 'fname' ,'stim1']).min()
vep1 = vep1.reset_index()


# vep2 = _inp[(_inp.times>0.6) & (_inp.times<0.7)]
# vep2 = vep2.ix[vep2.groupby(['stim1' ,'fname']).lfp.idxmax().values]
# vep2 = vep2.groupby( ['stim1', 'fname']).max()
# vep2 = vep2.reset_index()
# vep2['vep'] = 'dd'

# vep3 = _inp[(_inp.times>0.93) & (_inp.times<1.03)]
# vep3 = vep3.ix[vep3.groupby('fname').lfp.idxmin().values]
# vep3 = vep3.groupby('fname').min()
# vep3['vep'] = 3
colors = my_pal[::2]

vep1 = vep1.sort_values(by=['stim1'])
# _data = band_df
g = sns.factorplot(x="stim1", y="lfp",  data=vep1,  kind = 'bar', ci = 68, palette = colors,
#                     capsize = 0.02, saturation = 1,
                   size = 5, aspect = 1 )
# plt.ylim(-700, 700)
# plt.savefig('deep_cir_vep_bar.pdf', transparent=True)
# plt.savefig('sert_ko_amp_vep_all.png', transparent=True)


In [ ]:
x1 = vep1[vep1.stim1 == 1].lfp.values
x2 = vep1[vep1.stim1 == 2].lfp.values
x3 = vep1[vep1.stim1 == 3].lfp.values
x4 = vep1[vep1.stim1 == 4].lfp.values

print np.mean(x1), sstat.sem(x1)
print np.mean(x2), sstat.sem(x2)
print np.mean(x3), sstat.sem(x3)
print np.mean(x4), sstat.sem(x4)

print len(x1), len(x2), len(x3), len(x4)
print sstat.kruskal(x1, x2, x3, x4)

print len(x1)

print sstat.mannwhitneyu(x1, x2)
print sstat.mannwhitneyu(x1, x3)
print sstat.mannwhitneyu(x1, x4)
print sstat.mannwhitneyu(x2, x3)
print sstat.mannwhitneyu(x2, x4)
print sstat.mannwhitneyu(x3, x4)

### TF analysis

In [ ]:
import OpenOE_AC_map_functions_v1_08_30s as oem
from  __builtin__ import any as b_any
import matplotlib.ticker as tkr

In [ ]:
ls = []
ls2= []
ls_perm = []
sf_col = [0.01, 0.02, 0.04, 0.08, 0.14 ]
_data = _inp[(_inp.times > 0.1) & (_inp.times < 1.9)]
#               & (sert_df.stim1 == '1')
#               & (sert_df.stim2 == 'G')
#                & (sert_df.times < 2.5)
               #]

param = 'stim1'
n =sorted(_data[param].unique()) 
f, ax = plt.subplots( 1, len(n), sharex=True, sharey=True, figsize=(12,6), facecolor = 'w')
t1, t2 = 350, 550


for idx, val in enumerate(sorted(_data[param].unique())):
    
    #tmp = _data[_data['stim1']==sf].pivot_table(index = ['fname', 'et'], columns= 'times' ,values = 'lfp').mean(level=1)
    tmp = _data[_data[param]==val].pivot(index = 'fname', columns= 'times' ,values = 'lfp') 
 
    tf, time, frex, tf3d = oem.tf_cmw(ax[idx], tmp, show=True, log_scale=False)
    ax[idx].set_title('' + str(val))
    
    tf3_arr = np.dstack(tf3d)
    tf3_arr = 10*np.log10(tf3_arr/tf3_arr[:,:300,:].mean(axis=1, keepdims=1))

#     mask = frex[(frex>12) & (frex<25)]
#     ax[idx].plot(((tf[1, int(mask[0]):int(mask[-1]), :]).mean(axis=0)))
    ls_perm.append(tf3_arr)
    
    tmp = np.dstack(tf3d)
    tmp = tmp.swapaxes(0,2)
    tmp = 10*np.log10(tmp/tmp[:,:300,:].mean(axis=1, keepdims=1))

    alpha = np.mean(tmp[(frex>8) & (frex<=12)], axis=0)[t1:t2]
    alpha = np.mean(alpha,axis=0)

    beta = np.mean(tmp[(frex>12) & (frex<=25)], axis=0)[t1:t2]
    beta = np.mean(beta,axis=0)

    low_gamma = np.mean(tmp[(frex>30) & (frex<=50)], axis=0)[t1:t2]
    low_gamma = np.mean(low_gamma,axis=0)

    high_gamma = np.mean(tmp[(frex>50) ], axis=0)[t1:t2]
    high_gamma = np.mean(high_gamma,axis=0)

    theta = np.mean(tmp[(frex>=4) & (frex<=8)], axis=0)[t1:t2]
    theta = np.mean(theta,axis=0)


    tf_tmp = pd.DataFrame({'cond': val,  'beta':beta, 'hg': high_gamma,
                           'alpha': alpha,   'lg': low_gamma, 'theta': theta,
                   'index': np.arange(0, _data[_data[param]==val].fname.unique().size)
                     })
    ls2.append(tf_tmp)
# tf_out = pd.concat(ls2)
# tf_tmp = pd.melt(tf_out, id_vars=['cond'], value_vars=['theta', 'alpha', 'beta', 'lg','hg' ])
# tf_out = pd.melt(tf_out, id_vars=['cond','index' ], 
#                  value_vars=['theta', 'alpha', 'beta',  'lg','hg' ])
# tf_out = tf_out.groupby(['cond','variable', 'index']).mean().reset_index()
plt.tight_layout()

# plt.savefig("tf_l4_kic.png", transparent=True)
# plt.savefig("deep_tf_kic.pdf", transparent=True)

In [ ]:
import mne
from mne.stats import ttest_1samp_no_p
from functools import partial

In [ ]:
sigma = 1e-3
threshold_tfce = dict(start=5, step=1)
stat_fun_hat = partial(ttest_1samp_no_p, sigma=sigma)
T_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_1samp_test(ls_perm[0],
                             n_permutations = 100, threshold = 7.0,  tail = 1, t_power=1)

T_obs_plot = np.nan * np.ones_like(T_obs)
for c, p_val in zip(clusters, cluster_p_values):
    if p_val <= 0.05:
        T_obs_plot[c] = T_obs[c]

# f, ax = plt.subplots(figsize = (8,2.5))
# ax.set_xlim(0,2.5)
# ax.set_yscale('log')
# hm = ax.set_yticks(np.logspace(np.log10(2),np.log10(80),6))
# ax.set_yticklabels(np.round(np.logspace(np.log10(2),np.log10(80),6)))
# ax.contourf(time, frex, T_obs.T, 
           
#            aspect='auto', origin='lower', cmap='gray')
# hm = ax.contourf( T_obs_plot.T, 
           
#            aspect='auto', origin='lower', cmap='jet')
# plt.colorbar(hm)
# plt.xlabel('Time (s)')
# plt.ylabel('Frequency (Hz)')
# plt.savefig("tf_permutation.png", transparent=True)
# plt.savefig("deep_tf_perm_kic_rot.pdf", transparent=True)

In [ ]:
f, ax = plt.subplots(figsize = (2.8,5.6))
hm = ax.contourf( time, frex, T_obs_plot.T, 
           
           aspect='auto', origin='lower', cmap='jet')
# plt.savefig("tf_perm_line.pdf", transparent=True)

## Generate CSD pkl file

In [ ]:
ls2 = []

times = np.linspace(0, 1.0, 1000)
f, ax = plt.subplots(2, figsize = (1,1))
for fname in matches:
    if '201' in fname and 'CH65_2' in fname:
        continue
    else:
        print fname
#     t = fname.split('\\')[-1].split()[-1].split('_')[4]
#     if b_any(t in x for x in good_recs.fname.unique()):
#         print fname
#     else:
#         continue
    
    et = fname.split('\\')[-1].split()[0][1:]
    rec = fname.split('\\')[-1].split()[1].split('_')[0]
    paradigm = fname.split('\\')[-1].split()[-1].split('_')[-6]
    exp =  fname.split('\\')[-1].split()[-1].split('_')[1]
    
    ls = []
        
    try:
        tmp =  pd.read_hdf(fname, key = 'raw') 
    except:
        continue
    trials =  int(tmp.shape[1]/1000)


    csd_tmp = np.split(tmp, 200, axis = 1)
    csd_input = np.dstack(csd_tmp)
    

    for i in np.unique(opto_seq):
        if i == 0:
            continue
        for j in np.unique(trial_seq):
        
            sel_idx = np.where((opto_seq == i) & (trial_seq == j))[0]
            csd_tmp = csd_input[:,:, sel_idx].mean(axis = 2)

            csd_tmp = pd.DataFrame(csd_tmp)
            csd_tmp = csd_tmp - csd_tmp.median(axis = 0)
            ddf2 = csd_tmp
            csd = oem.df_CSD_analysis( np.array(ddf2), ax, 
                              Channel_Number=np.shape(ddf2)[0], show_plot=False)
            tmp_df = pd.DataFrame(csd).stack().reset_index()
            tmp_df.columns = ['csd_step', 'samples', 'csd']
            tmp_df['et'] = et
            tmp_df['exp'] = exp
            tmp_df['paradigm'] = paradigm
            tmp_df['rec'] = rec
            tmp_df['stim1'] = j
            tmp_df['opto'] = i
            tmp_df['fname'] = fname
            tmp_df['times'] = tmp_df['samples']/1000
            ls.append(tmp_df)
    tmp_df2 = pd.concat(ls)
    ls2.append(tmp_df2)
kic_csd = pd.concat(ls2)
    

In [ ]:
kic_csd.groupby('fname').csd.min()

In [ ]:
_data = kic_csd[kic_csd.et.str.startswith('4')]
n = sorted(_data.stim1.unique())
f,ax = plt.subplots(len(n),1, figsize = (8,12), sharey= True, sharex=True,facecolor = 'w')
cbar_ax = f.add_axes([.97, .3, .03, .4])
formatter = tkr.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-2, 2))


for idx, val in enumerate(n):
    hm_input = _data[_data.stim1 == val].groupby(['csd_step', 
            'times']).mean().reset_index().pivot('csd_step', 'times', 'csd').values
    hm = sns.heatmap(hm_input, cmap = 'jet',  annot=False, xticklabels=  500, ax = ax[idx], 
                cbar = 0==idx, 
                cbar_kws={ "format": formatter},
                vmin = -1e6, vmax = 1e6, robust = True, yticklabels=False,  cbar_ax=None if idx else cbar_ax )
    ax[idx].invert_yaxis()
    ax[idx].set_title(val)
# mappable = hm.get_children()[0]
# plt.colorbar(mappable, ax = [ax[0],ax[1], ax[2], ax[3]],orientation = 'horizontal')
# plt.savefig("csd_kic.png", transparent=True)

In [ ]:
path = r"U:\Data_Analysis\pak6\Analysis of units\kic2_archt\kic-archt-csd-all-recs-01.pkl"
kic_csd = pd.read_pickle(path)

In [ ]:
mmn_exp_csd.to_hdf('csd_column01_kic.hdf','data',mode='w')

In [ ]:
_data = mmn_exp_csd[(mmn_exp_csd.opto == 0) & (mmn_exp_csd.stim1 == 4)]
ls = []
for fname in _data.fname.unique():
    out = _data[_data.fname == fname].reset_index().pivot('csd_step', 'times', 'csd').values
    ls.append(out)
csd = np.dstack(ls)
csd4 = np.swapaxes(csd,0,2)


In [ ]:
csd2 = np.swapaxes(csd2,0,2)

In [ ]:
fname.split('\\')[-1].split()[1].split('_')[4]